In [ ]:
!pip install openai
!pip install boto3

In [ ]:
from openai import AzureOpenAI
import boto3
import json
from botocore.exceptions import ClientError
import time

In [ ]:
def call_azure_openai(prompt):
    client_openai = AzureOpenAI(
        api_key="",
        api_version="",
        azure_endpoint=""
    )

    response = client_openai.chat.completions.create(
        model="",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2048
    )
    return response.choices[0].message.content

  #max input = 128,000
  #max output = 16,384

In [ ]:
def call_llama3_1_aws(prompt):
  aws_region = ''
  access_key_id = ''
  secret_access_key = ''

  # Create a Bedrock client
  bedrock = boto3.client(
    '',
    region_name=aws_region,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key
  )

  model_id = ""
  native_request = {
    "prompt": prompt,
    "max_gen_len" : 2048  #MAX allowed by AWS
  }

  request = json.dumps(native_request)
  try:
    response = bedrock.invoke_model(modelId=model_id, body=request)
    response_body = json.loads(response.get('body').read())

  except ClientError as e:
    # Handle any errors that occur during the invocation
    print(f"An error occurred: {e.response['Error']['Message']}")

  return response_body.get('generation')

In [ ]:
def count_tokens(text):
    # Approximation: 1 token ~ 4 characters, adjust as necessary
    return len(text) // 4 + (len(text) % 4 > 0)  # Round up for any remaining characters

def evaluate_models_with_prompts(prompts):
    results = []

    for prompt in prompts:
        # Measure Azure OpenAI
        start_time = time.time()
        azure_output = call_azure_openai(prompt)
        azure_time = time.time() - start_time
        azure_characters = len(azure_output)
        azure_tokens = count_tokens(azure_output)  # Count tokens in the output

        # Measure Llama 3 on AWS
        start_time = time.time()
        llama_output = call_llama3_1_aws(prompt)
        llama_time = time.time() - start_time
        llama_characters = len(llama_output) if llama_output else 0
        llama_tokens = count_tokens(llama_output)  # Count tokens in the output

        # Store results
        results.append({
            'prompt': prompt,
            'input_characters': len(prompt),  # Count characters in the prompt
            'input_tokens': count_tokens(prompt), # Count tokens in the prompt
            'azure_time': azure_time,
            'azure_characters': azure_characters,
            'azure_tokens': azure_tokens,
            'llama_time': llama_time,
            'llama_characters': llama_characters,
            'llama_tokens': llama_tokens,
        })

    return results

In [ ]:
# Example prompts for testing
prompts_to_test = [
    # 1st prompt with approximately 1000 input tokens
    (
        """Summarize the following story - In the heart of the enchanted kingdom of Eldoria, there lay a village called Oakshade. This village was known for its lush, green trees and sparkling rivers, which glimmered like diamonds in the sunlight. However, unbeknownst to its inhabitants, a mysterious darkness was beginning to envelop their land.

One sunny morning, a young woman named Elara woke up with a sense of unease. Ever since she had heard the whispers of the townsfolk about a Shimmering Stone hidden deep within the Forest of Shadows, she couldn’t shake the feeling that destiny was calling her. The Shimmering Stone was said to hold immense power, capable of either saving the kingdom or bringing about its downfall, depending on who wielded it.

Elara had always been adventurous, often exploring the edges of the Forest of Shadows that bordered their village. Despite the dark rumors that surrounded it, she had collected beautiful flowers and rare herbs from the forest to help her family and neighbors. But now, a heavy heart weighed on her as she recalled stories from the elders about the stone’s significance. They spoke of its glow, which once filled the entire kingdom with light and warmth. Elara felt compelled to uncover the truth.

Determined, she packed a small satchel with supplies: dried bread, a waterskin, her trusty dagger, and a worn map handed down from her grandmother. The map promised to lead her to the heart of the forest, where the Shimmering Stone lay hidden. Just as she was about to leave, her younger brother, Finn, rushed to her side, his blue eyes filled with excitement.

“Let me come with you, Elara! I can be brave too!” Finn pleaded, bouncing on the balls of his feet.

Elara hesitated. The journey would be perilous, but she couldn’t ignore the fire of determination in her brother’s eyes. After giving it a moment's thought, she knelt down and said, “Alright, but we must stick together and promise to return home.”

With a shared sense of purpose, the siblings ventured into the Forest of Shadows. The canopy overhead loomed dark and forbidding, with twisted branches that seemed to whisper secrets as they brushed against one another. As they walked deeper into the forest, the sunlight faded, replaced by a cool, eerie twilight.

After several hours of walking, they reached a clearing filled with shimmering blue flowers that seemed to glow. In the center of this natural oasis stood a tall fountain with crystal-clear water flowing from it. Elara remembered the tales of the Fountain of Reflection, said to reveal one’s true feelings when gazed upon.

“Should we look?” Finn asked, curiosity piqued.

Elara nodded and leaned over the fountain’s edge, gazing into the water. Instead of her reflection, she saw herself standing before the Shimmering Stone, its light illuminating a path forward. But the image quickly shifted, revealing shadows chasing her and Finn, darkness closing in.

“I... I don’t think we should look too long,” Elara said, pulling Finn away. The siblings breathed heavily, both feeling the weight of what they had seen.

Suddenly, a rustling sound startled them, and a figure emerged from the bushes. It was an old woman with a crooked back and a long, flowing robe made from leaves and vines.

“Seekers of the Shimmering Stone, why do you trespass in these woods?” she croaked, her voice both intimidating and gentle.

“We seek the Shimmering Stone to save our village from darkness,” Elara replied bravely, her heart racing. “Can you help us?”

The old woman studied them for a moment, her gray eyes sparkling with intrigue. “Many have sought the stone, but only those with pure hearts may claim its power. You will face challenges that will test your courage and bond as siblings. Only then will the stone reveal itself.”

With a wave of her hand, the old woman conjured a map of the forest, illustrating all the hazards ahead. “Beware of the shadow creatures that await. They feed on fear and doubt. Trust in each other, and remember, the light of hope will guide your way!”

Thanking the woman, the siblings pressed forward, clutching the map tightly. Soon, they encountered their first challenge: a vast chasm that seemed to stretch infinitely below. The only way across was a series of narrow, swaying bridges suspended high above the abyss.

“Do you think we can cross these?” Finn asked, his voice shaking.

“I know we can,” Elara encouraged. “Just hold my hand and follow my lead.” Taking a deep breath, they grasped each other’s hands and stepped onto the first bridge. The wooden planks creaked dangerously, and Finn’s knees felt weak. Panting, he looked at Elara, who smiled reassuringly.

“Keep going,” she said. “Just focus on me.”

With each step, they found their rhythm. The sway of the bridge felt less daunting with their combined strength. Finally, they reached the other side, where they collapsed in a mix of relief and laughter.

“See? We did it!” Elara exclaimed.

But their joy was short-lived. As they continued on their path, the air thickened with an overwhelming darkness, and shadow creatures emerged from the trees. They were ghostly figures, their forms swirling and flickering like smoke.

“Listen to the whispers of doubt,” the shadows hissed. “You’ll never find the stone. You’re just weak children!”

Finn trembled, fear gripping his heart. “What do we do, Elara?”

“Remember what the old woman said! They feed on fear,” Elara whispered fiercely, trying to suppress her own rising anxiety. “We have to stay strong and believe in each other.”

Drawing strength from her brother, she faced the shadows. “You’re wrong! We have the courage and love that no darkness can extinguish!” she shouted, her voice rising above the whispers.

Unexpectedly, a brilliant light erupted from their joined hands, pushing back the shadows. The creatures shrieked, recoiling as if struck by a powerful force. The siblings tightened their grip, feeling the warmth of hope envelop them.

With the shadows dissipated, they pressed on, energized by this victory. Hours later, they finally approached an ancient stone temple flanked by two massive statues of guardians. The air shimmered with magic, and in the temple’s center, they saw it—the Shimmering Stone, glowing like a thousand stars.

“It’s beautiful!” Finn gasped, his eyes wide with wonder.

As they stepped closer, the stone pulsed with energy, illuminating the room. But before they could touch it, a voice echoed through the temple. “Only those who truly understand the meaning of sacrifice may claim the stone.”

Elara and Finn looked at each other, puzzled but determined. “What do we need to sacrifice?” Elara asked.

“Prove your love for one another,” the voice continued. “Only by placing what you cherish most on the altar may the stone be yours.”

Elara’s heart sank. Both she and Finn were aware of their deep bond, but this was a test unlike any other. Finn hesitated, clutching his satchel, filled with a few cherished trinkets from their childhood. Meanwhile, Elara held onto a locket of their late mother—something she had always kept close to her heart.

“I can’t give up Mom’s locket!” Elara cried. “It’s all I have left of her!”

Finn nodded, understanding the weight of her words. “And I can’t part with my toys, Elara. They remind me of the good times we had!”

“Maybe…” Elara’s voice trailed off. “Maybe it’s not about giving up our most cherished things, but understanding the value of love and courage we share.”

Taking a deep breath, Finn stepped forward, placing his satchel on the altar. “I would trade my favorite things any day for our family’s safety.”

Moved by his selflessness, Elara followed suit, placing her locket beside his satchel. They both stepped back, watching as the stone blazed brightly, lighting up the entire temple. With a hum of energy, the stone floated closer, enveloping them in a warm, golden light.

In that moment, they knew they had passed the test. The light faded, and the Shimmering Stone glided down to them, resting softly in their hands. It shone radiantly, as if the warmth of love had been infused into its very essence.

“You have proven yourselves to be worthy,” the temple echoed. “Together, you shall harness the stone’s power to heal your kingdom.”

Armed with the stone, Elara and Finn exited the temple, their hearts filled with hope. The darkness that loomed over Oakshade had vanished, replaced by the brilliant light radiated by the Shimmering Stone.

As they returned home, the villagers gathered around, amazed at their bravery. With the Stone’s power, Elara and Finn restored light and prosperity to their homeland, their bond now stronger than ever.

From that day forward, the tale of Elara and Finn became legend in Eldoria, a reminder that courage, love, and sacrifice could conquer the deepest darkness. "
"""
    ),

    # 2nd prompt with approximately 10,000 input tokens
    (
        """Summarize the following story - In the faraway kingdom of Valoria, where the sun kissed the earth with golden rays and the hills danced with wildflowers, there lived a legendary knight named Sir Cedric. His deeds were sung by bards in every tavern, and children idolized him as a hero, but few knew the true weight of his responsibilities. Sir Cedric was not only known for his bravery but also for the kindness that flowed from his heart like the rivers that nourished the land.

One day, as Cedric gazed out from his castle window, a sense of foreboding nestled in his chest. The horizon, usually vibrant with life, appeared dimmer. The villagers had been unusually quiet, their laughter replaced by whispers filled with worry. Without delay, Cedric donned his armor, polished and gleaming as bright as the morning sun. It was time to uncover the source of this unease.

Riding through the village, Cedric was greeted by filled teeming streets lined with colorful stalls. The scent of bread wafted through the air, mingling with the earthy aroma of fresh hay. Yet, the joy that once filled the hearts of the villagers seemed stifled, weighed down by an invisible force. As he approached the central square, he noticed old Mrs. Hargrove sitting by her stall, her hands trembling as she knitted.

“Good day, Mrs. Hargrove,” Cedric called cheerily. “What troubles you on this fine morning?”

She looked up at him with weary eyes. “Oh, Sir Cedric, the crops have failed once again, the livestock are restless, and the river’s flow has dwindled! We fear there may be dark forces at work, and people are becoming desperate.”

Cedric's heart sank. The land of Valoria thrived on its crops and rivers, and he knew desperation could drive even the gentlest soul to madness. “Fear not, good lady. I will investigate this matter and bring back prosperity for our home,” he assured her, determination burning in his chest.

As he rode toward the Eldwood Forest, the shadows grew longer, and a dense fog began to loom. Eldwood was both beautiful and treacherous, known for its towering trees and sparkling rivers but also for the legends that spoke of ancient magic and creatures that roamed its depths. The further Cedric ventured, the more he felt a presence, as if the trees themselves were watching him with bated breath.

Passing through the forest's edge, Cedric's keen eyes caught sight of a flickering light in the distance. Curiosity piqued, he ventured closer. As he approached, he stumbled upon a clearing where a brilliant azure flame danced upon a stone pedestal. Atop it rested a crystal orb, shimmering with an otherworldly radiance.

“Mystical enchantment,” he murmured, reaching out towards it, but a voice stopped him in his tracks.

“Only those pure of heart may take from the Keeper of the Flame,” the voice echoed, resonating through the clearing. From the shadows emerged a figure – a woman draped in robes that glimmered like stars, her hair flowing like rivers of light.

“I am Aurelia, Guardian of the Eldwood,” she proclaimed. “Many have sought my orb, believing it to contain the answers to their woes. But its power comes with a price.”

“What price?” Cedric inquired, his mind racing. He needed to understand if this orb held the salvation his people sought.

“To wield the orb, you must confront the truths within your soul, face the shadows of your past, and emerge whole. Only then can it grant you the strength to combat the forces that plague Valoria,” Aurelia explained, her eyes piercing yet warm.

Noble and brave as he was, the notion of introspection filled Cedric with doubt. He had spent years combating foes in physical battles, but the battle of the soul was one he had never faced. Still, he understood the consequences of inaction and voiced his intent. “I will confront whatever lies within. I owe it to my people.”

“Very well,” Aurelia said, her voice steady. “Approach the orb and lay your hand upon it.”

Cedric stepped forward, heart pounding, and placed his hand on the cool surface of the orb. Suddenly, a surge of energy coursed through him, blinding him in a flash of light. When he opened his eyes, he found himself standing in a long corridor dimly lit by flickering torches. The air was heavy, filled with the whispers of the past.

“Welcome to the Realm of Reflection,” the voice of Aurelia echoed through the corridor. “Here, you shall face your memories, your regrets, your greatest fears.”

Without warning, scenes began unfolding around him. The first memory to emerge was from his childhood: a young Cedric stood before a large wooden door, the sounds of laughter echoing behind it. His father was hosting a grand feast, but Cedric, shy and uncertain, refrained from joining the celebration.

“Why didn’t you ever join them?” Aurelia’s voice was now a soft whisper, coaxing him to confront his choices.

“I felt like an outsider,” he replied, struggling with the emotions from that long-forgotten day. “I was scared of not being accepted.”

The memory shifted, showing him the day he was knighted. Cedric stood proudly in shining armor, but his eyes betrayed the burden of expectations resting on his shoulders. He saw his mentors smiling, the townspeople cheering, yet he felt an irrepressible fear – the fear of failing them, of not living up to the title of knight.

“Your fears are a part of you, Sir Cedric,” Aurelia said. “But they do not define you. You must understand that true courage lies within acceptance, not just of yourself but also of the vulnerability that comes with it.”

With each passing memory, Cedric faced moments of doubt, anger, and grief. He relived significant failures—the times he had let his friends or fellow knights down. Each memory warped into a haunting specter, whispering words of defeat, urging him to surrender.

But through the darkness, a flicker of light emerged with each confrontation. With each moment faced, he felt a weight lift as he embraced his flaws, realizing they were not signs of weakness but colors on the canvas of his life.

Finally, he found himself standing before a great mirror, its surface dark and murky. “To finish this journey, you must look into the mirror and accept who you are,” Aurelia guided him.

Taking a deep breath, Cedric stepped forward. As he gazed into the depths of the mirror, his reflection transformed. He saw himself not only as a knight but as a man, vulnerable yet strong, filled with untold scars but also untapped potential. The mirror shimmered, encapsulating his past, present, and future in a whirlwind of light.

“I am Sir Cedric, a knight of Valoria. I embrace my past and the fears that once confined me,” he declared, voice steady. “I accept my flaws and care for those I protect. I fight not only for honor but for love and hope. My journey has shaped me.”

At that moment, the mirror exploded into a cascade of colors, enveloping him in a warm glow. He felt renewed, empowered, and utterly freed. The realm began to dissolve around him, and with one final surge of energy, Cedric found himself back in the clearing, the orb still glowing on the pedestal before him.

“Congratulations,” Aurelia smiled softly. “You have faced your shadows and emerged stronger. You now hold the strength to protect Valoria.”

With newfound determination, Cedric lifted the orb and felt its power resonate through him. “Thank you, Aurelia. I will use this gift wisely,” he vowed.

With a gentle nod, Aurelia stepped back into the shadows of the Eldwood, leaving Cedric to navigate his way back to his village. As he returned, the skies began to clear, and the sun's warmth enveloped him, bringing back life to the flowers and trees.

The villagers marveled at his return, noticing the light emanating from him as if he had absorbed the essence of the sun. Cedric gathered them in the square, feeling the weight of their hopeful gazes upon him.

“My friends! I have faced a great trial and discovered the true strength that lies within us all!” he called out, the orb glowing beside him. “Together, we can overcome the darkness that has crept into our land. This orb will help us restore balance—the crops will flourish again, and peace shall reign!”

As Cedric spoke, the orb pulsed, sending waves of radiant light across Valoria. Crops began to bloom, the river’s flow surged, and laughter erupted once more among the villagers. Hope returned, stronger than ever.

Under Cedric’s vigilant watch, he led initiatives to support and uplift the villagers. They started a harvest festival, where everyone came together to celebrate not just the bounties of the land but the unity they had fostered through their struggles. The air was filled with music, laughter, and camaraderie as old wounds healed and new friendships blossomed.

Cedric often found children gathering around him, eager to hear tales of his adventures, but this time, he did not only recount battles but also spoke of vulnerability, courage, and the true essence of bravery that lay in accepting oneself.

As seasons turned, the kingdom flourished, yet Cedric’s desire to protect Valoria never waned. He knew whispers of darkness still roamed the far corners of the kingdom. So, he established a council, training young knights not only in combat but in wisdom and leadership.

Together, they spent hours discussing the meaning of true strength and the importance of embracing their fears. Cedric’s favorite was a young girl named Lyra, who possessed an uncanny talent for archery. She dreamt of becoming a knight and would often shadow Cedric, soaking in every lesson he shared.

One evening, as they practiced their skills, Lyra confessed, “Sometimes, Sir Cedric, I doubt myself. I’m afraid I’ll never be as good as you or the others.”

Cedric paused, seeing the same hesitation in her eyes that he once harbored. “Lyra, the journey to strength is not always about being the best. It’s about facing your doubts and nurturing your passion. Embrace who you are and cherish your journey. You are capable beyond your belief.”

In time, Lyra became one of Cedric’s most trusted knights, embodying the spirit he had instilled in her. They fought alongside each other, not just against external threats but for the wellbeing of those they loved.

But change often comes with trials. Word began to spread of a rising shadow from the North, a dark sorcerer named Malakar who thrived on chaos and despair. It was said that he sought to harness the ancient magic buried deep within Eldwood, forcing the unkind energies of despair over those who opposed him.

Determined to protect Valoria, Cedric gathered his knights and ventured into the distant lands to confront Malakar. The journey was fraught with peril, as they traversed mountains, crossed fierce rivers, and battled relentless storms. Each night around the campfire, Cedric reminded his comrades of their purpose—the love they shared for their land and each other.

Finally, they arrived at the fortress where Malakar resided, shrouded in a perpetual storm. The laughter of Valoria seemed a distant memory amidst the oppressive darkness. Cedric and his knights stood firm, ready to face what was to come.

They stormed the gates, swords gleaming in the faint light, undaunted by the specters that wailed in torment around them. Within the fortress, Malakar awaited, cloaked in shadows, his presence chilling the very air.

“Brave knights,” he mocked. “What brings you to my domain? Do you seek glory? Or do you simply wish to meet your end?”

Cedric stepped forward, unwavering. “We bring hope! Your darkness will not prevail over the light that bonds Valoria!”

With a wave of his hand, Malakar unleashed a wave of dark energy, sending Cedric and the knights sprawling. The shadows twisted and writhed, threatening to engulf them. But Cedric stood, fueled by the love and hope of his people.

Calling forth the orb’s power, Cedric raised it high, its azure flame igniting the battlefield. Surrounding him, his knights rallied, pushing against the dark tide. Lyra, by his side, loosed her arrows that glimmered like stars, striking the shadows with precision.

Together, they advanced towards Malakar, the orb illuminating the darkened fortress. The sorcerer, realizing his impending defeat, unleashed waves of despair, flooding the air with nightmares.

But Cedric remembered Aurelia’s words—the key was unity. Amidst the chaos, he raised his voice. “Remember the light within! Together, we are stronger than his darkness!”

Lyra ran to Cedric’s side, her courage shining as bright as the orb. The other knights joined, forming a circle of unity around Cedric. They focused their hearts and souls into the orb, channeling their fears and hopes, causing it to burst into blinding light.

The darkness faced a relentless tide, pushing back the shadows. With a powerful surge, they connected their spirits, empowered by the love that resided in their hearts. The orb erupted, sending beams of light cascading throughout the fortress, obliterating Malakar’s malevolence.

Screaming in fury, Malakar found his power waning, shadows seeping away. Realizing he had lost, he attempted to retreat, but it was too late. The light engulfed him, and with a final cry, he vanished into the ether, the fortress crumbling around them.

As silence fell over the fortress, Cedric and his knights emerged victorious. They had not only defeated the darkness but had illuminated a path of hope and belonging that would resonate across Valoria.

Returning home, they were met with joyous celebrations. The people of Valoria rejoiced, grateful for Cedric and his knights. But Cedric knew it was not just him—it was the unity, love, and acceptance they had nurtured.

In the many years that followed, Cedric continued to serve as a guiding light for his people. He ensured the teachings of courage, vulnerability, and unity were carried forward. The stories of their triumph over darkness turned into legends, inspiring generations to come.

His heart swelled with pride each time he recounted his journey alongside loyal knights like Lyra, and they forged a new legacy together where darkness could never extinguish the vibrant light of Valoria.

And so, the kingdom flourished under Cedric’s watchful eye, enveloped in peace and prosperity. Cedric never forgot Aurelia’s wisdom. He embraced every moment, knowing that true bravery lay not in the absence of fear, but in the courage to face it, hand in hand with those they loved.

In the quiet evenings, under the watchful stars, Cedric knew that a hero was not merely one who wielded a sword, but one who nurtured the light within every heart. And as the people of Valoria looked upon him with reverence, a new legend began—a legend not of a knight, but of a man who loved fiercely, dreamed boldly, and conquered darkness with the radiant power of hope.
As the celebrations in Valoria thrived, peace reigned for many moons, yet Cedric knew that the echoes of Malakar’s darkness still lingered in the realms beyond their borders. Though the kingdom flourished, he felt a whisper within him that the threat was not entirely vanquished. Strange rumors started creeping into the village: sightings of shadowy figures moving under the cover of night and villagers speaking of nightmares that seemed more vivid than before.

It wasn’t long before a council was convened to discuss the emerging unease. Cedric, accompanied by his trusted knights, seated himself at the round table in the grand hall of Eldoria Castle. Lyra sat beside him, exuding confidence and fire, her eyes reflecting her unwavering determination.

As the council gathered, Elder Garrick, the wisest among them, stood to address the assembly. “Our kingdom faces shadows anew. We must not allow complacency to dull our senses. The essence of Malakar might still corrode the edges of our peace.”

Cedric nodded gravely. “Elder Garrick speaks truthfully. If remnants of Malakar's power linger, we must seek them out, lest they fester into something much worse. I propose we dispatch scouting parties to investigate these claims and gather intelligence,” he suggested.

Lyra raised her hand, her expression fierce. “I wish to lead one of the scout parties, Sir Cedric. I’ve trained hard, and I believe I can uncover the truth of these shadows!”

“Bravery shines bright within you, Lyra. However, I must consider the dangers,” Cedric replied, appreciating her fervor.

“Cedric,” she insisted, “we’ve faced darkness before. I can handle this! I will not be a damsel in distress; I have proven myself time and time again. Trust me!”

The knight smiled, feeling pride swell in his chest. “Very well. I trust your instincts. Gather volunteers, and prepare a team. We shall move by dawn.”

As the council dispersed, Cedric remained deep in thought. He could feel shadows gathering on the horizon. He resolved to join Lyra on her scouting mission. Together, they would face whatever challenges awaited them, supporting each other as they always had.

The sun rose over Valoria, painting the sky with hues of orange and pink. As the knights assembled, Cedric and Lyra led the charge out of Eldoria Castle, their hearts united with purpose. The group rode through the fields and into the woods, where the shadows thickened, and a chill settled in the air.

The group’s journey was cautious but composed. They navigated through the dense trees and underbrush, keenly aware of their surroundings. After hours of riding, they reached a hidden clearing where they found remnants of a gathering. The ground was scorched, charred markings sprawling across the earth, reminiscent of dark magic.

Lyra knelt to examine the ground. “This was recently conjured. The darkness has a lingering essence—something evil transpired here,” she said, a frown creasing her brow.

As they continued their investigation, the sounds of distant murmurs permeated the air. With a gesture, Cedric signaled the knights to silence. They remained hidden behind the trees as a group of figures came into view, clad in ragged robes infused with dark energy. Cedric recognized their insignia—a shadowy emblem representing Malakar.

“Gather, my minions!” a figure commanded, shrouded in darkness. “We will harness the fears they leave unguarded, and the remnants of our master will grant us the power to rule Valoria once more!”

Cedric's heart raced as anger boiled within him. “These are remnants of Malakar's followers. They must not be allowed to gather strength!” he whispered urgently.

Lyra nodded, determination burning in her eyes. “We have to strike before they can amplify their dark energy. We cannot allow this to fester!”

Cedric formulated a battle plan. “We’ll create a distraction. My knights will provide cover while you and I flank from the east. We will catch them off guard!”

The knights mobilized quickly. With a battle cry, they charged, drawing the attention of Malakar’s followers. As the shadowy figures turned, Cedric and Lyra slipped around the flank, darting into the chaos.

“Remember our training!” Cedric yelled to the knights as they engaged the enemies. The battle was fierce; shadows clashed with shining steel, and the air crackled with magic.

As Cedric confronted the leader of the dark followers, his sword gleamed in the smoky air. “You shall not use this land for your darkness!” he declared, charging forward.

The figure cackled, summoning dark tendrils from the ground that lashed out at Cedric. He ducked and rolled, narrowly avoiding the attack. “Foolish knight! You think you can take on the shadows that serve a master?”

“Your master is defeated! The light will always overcome the dark!” Cedric shouted, channeling strength from within as he surged forward, sword striking true.

Lyra fought alongside him, her arrows striking with precision, illuminating the battlefield with their brilliance. As their enemies fell one by one, the tide of battle turned in favor of the knights. The shadows began to retreat, fear written across their darkened faces.

But the leader of the followers remained resolute. “You think you have won? You will never know true despair!” he growled, drawing forth an amulet that glowed ominously.

At that moment, a powerful wave of dark magic exploded from the amulet, blasting Cedric and Lyra across the clearing. The force knocked them to the ground, leaving them dazed and disoriented.

Cedric struggled to his feet, panic rising as he saw the shadows converging again. They were regrouping, preparing for one last attack. “We must not falter!” he urged Lyra, adrenaline pumping.

“I’m with you,” she replied fiercely, her resolve crystallizing as she stood by his side.

Using the strength of their bond, they charged at the remaining shadows together, cutting through their ranks, their blades flashing in unison.

Amidst the chaos, Cedric noticed that the figure was chanting—a ritual. Realizing what was happening, he shouted, “Lyra, stop him! We must break his focus!”

Lyra nodded, drawing a deep breath as she nocked an arrow to her bow. With a steady hand and clear mind, she aimed and released, the arrow flying true. It struck the amulet, shattering the dark energy it had been channeling.

With a blinding flash, the magic exploded, enveloping the clearing in light. Cedric shielded his eyes, feeling the warmth wash over him as shadows dissipated into the ether. When the light faded, he saw the leader of the shadowy figures fall to the ground, defeated.

The remaining shadows fled into the forest, scattering like leaves in the wind. Victory was theirs, but the battle was not without cost; the darkness had left scars upon the landscape.

As Cedric and Lyra caught their breath, their knights surrounded them, praising their leadership and courageous hearts. “We did it!” one knight exclaimed, holding his sword high.

“Victory belongs to all of you,” Cedric said humbly, grateful for his comrades. “But we must uncover more of this threat. The darkness may return.”

Lyra glanced at the remnants of the amulet, flickering with residual magic. “We should take this back to Eldoria. Perhaps we can learn more about it and find a way to prevent this from happening again.”

The group began their journey back, weary yet triumphant. Upon their return to Eldoria Castle, they were greeted as heroes. The people gathered to hear tales of their bravery, their spirits reinforcing the bonds that held the kingdom together.

Gathering once more in the council chamber, Cedric laid the shattered amulet on the table. “This dark magic could pose a threat again. We cannot afford to let our guard down.”

Elder Garrick leaned forward, studying the fragments. “Ancient magic such as this can be difficult to eradicate. It thrives on fear and despair, and we must remain vigilant in our teachings to fortify our people’s hearts.”

With that, Cedric, Lyra, and the other knights devoted their efforts to educating the citizens of Valoria. They implemented training programs, teaching self-defense and resilience against dark influences while fostering a greater sense of community and unity.

Days turned into weeks, and hope began to flourish once more across the land. The shadows of the past began to lift, and the people, emboldened by their knightly protectors, stood stronger together.

However, amidst the renewed spirit, Cedric felt the weight of vigilance tugging at his heart. He often found himself gazing into the horizon, aware that darkness never truly vanished; it merely lay in wait.

One night, as Cedric prepared for bed, a knock echoed on his chamber door. It was Lyra, her eyes radiant with excitement.

“Cedric! I’ve come up with an idea,” she said, breathing heavily. “With the training programs yielding great results, I believe we can establish an Order of the Light, where we can gather knights and civilians alike to guard against any resurgence of dark magic!”

“An Order of the Light?” Cedric mused, intrigued. “What would it entail?”

“Imagine! A brotherhood and sisterhood bound by honor and courage, fostering hope and knowledge. We could train together, share wisdom, and rally against the loom of darkness! Our unity would be our greatest weapon!” she exclaimed passionately.

Inspired by her vision, Cedric nodded earnestly. “That is a brilliant idea, Lyra. We will need to hold a council to discuss this further.”

The very next day, assembling their most trusted knights and the villagers, Cedric and Lyra presented the concept of the Order of the Light. The response was overwhelming; excitement filled the air as people rallied around the prospect of joining forces for a unified cause.

Iterations of training began, and as months passed, the Order of the Light grew, welcoming all who sought to protect their homeland. Cedric and Lyra stood at the forefront, mentoring and inspiring others to face their fears and cultivate inner strength.

As the Order established itself, they created ceremonies to honor the virtues of bravery, unity, and resilience. Together, they held festivals celebrating the light within, invigorating the spirit of Valoria and solidifying their bonds.

But as the Order flourished, rumors began to circulate about the dark remnants that still lingered in the world. Across distant lands, whispers of a new dark sorceress, one who coveted the power of shadows, began to surface. The air grew thick with unease again, and the council knew they could not remain idle.

Cedric gathered his knights once more, preparing them for the daunting road ahead. “The shadows are shifting anew,” he stated gravely. “We must venture beyond our borders and confront this threat before it reaches our kingdom.”

Lyra stepped forward, driving her fist into her palm. “Let us not forget what we’ve accomplished; our strength is forged in unity! Together, we can face whatever shadows arise!”

Determined, Cedric prepared the horsemen, arming them with knowledge, courage, and new spells learned from the remnants of Malakar’s dark magic that were now theirs to wield wisely. He felt a renewed energy pulsate within him, ready to stand against whatever may come.

They set out at first light, crossing mountains, traversing rivers, and journeying over grasslands. Each knight felt a sense of purpose, honor, and pride, driven forth by their commitment to protect Valoria.

After several days of travel, they approached a valley overshadowed by looming peaks. The air felt heavy, and a chill crawled up Cedric’s spine. They continued onward, determination bolstering each step until they found a hidden entrance to a cavern shrouded in dark mist. This was the rumored lair of the dark sorceress.

“Stay alert,” Cedric commanded, his heart racing as they crossed the threshold into the darkness. The group moved through the winding tunnels, lit only by flickering torches that cast eerie shadows against the stone walls.

Deep within the cavern, they found her—a striking figure draped in flowing black garments that shimmered like the night sky. A crown of obsidian crystals adorned her head, reflecting the dim light and emitting a low hum. Her presence exuded power, a sense of menace that thickened the air.

“Welcome, knights of Valoria,” she said, her voice melodic yet chilling. “I am Selene, the Shadow Weaver. You’ve come seeking to reclaim your precious light, but you will find only despair!”

Cedric stepped forward, undeterred. “We shall not allow you to weave your dark magic into our lands! We stand united to protect the light that binds us!”

“Such sentiment!” Selene laughed, the sound echoing off the cave walls. “You cannot comprehend the beauty of the shadows. They wield power unimagined. Embrace the darkness, and you may yet survive!”

With a wave of her hand, shadows coiled like serpents, striking at Cedric and his knights. In response, they formed a protective circle, swords drawn, ancient spells surging as they fought against the encroaching darkness.

“Remember the light within you!” Cedric shouted, just as he felt the familiar warmth of hope envelop him. The orb from Eldwood responded, igniting with radiance, illuminating the cavern with a brilliant glow.

Selene recoiled, disbelief washing over her. “No! How dare you wield such power!” she exclaimed as she conjured a dark spell, swirling shadows spiraling into fierce winds.

The knights pressed forward, their swords glinting in the light as they cut through the shadows. Cedric, empowered by the light, charged straight at Selene, sword raised high. “This ends now!”

But just as he reached her, she unleashed a tempest of dark energy that sent him spiraling across the chamber. Collecting himself, Cedric spotted Lyra in a fierce duel of her own, dancing through the shadows like a spark of fire, each arrow finding its mark as she aimed at Selene.

“Ideals will not save you, Valorian knight!” Selene hissed, conjuring shadows around herself like a vortex.

Lyra met Selene's furious gaze, charging forward with tenacity. “Perhaps not, but the bonds between us are our greatest strength! Together, we can bring your darkness to an end!”

Cedric rallied, feeling the energy pulsing from the Order of the Light. The knights united, their hearts and souls intertwined. Together, they formed a shimmering barrier, pushing back Selene's tempestious magic.

“Your shadows cannot defeat our light!” Cedric declared, and with a unified cry, the knights charged, wielding both blades and magic, raging against the trembling darkness.

The struggle continued in an epic clash, each blow resonating with the values they cherished. Cedric pulled strength from the memories of past victories, the warmth of camaraderie fueling his every movement. The light from the orb flared with incredible intensity, engulfing the chamber in a radiant glow.

“This power belongs to those who embrace love, courage, and hope!” Cedric proclaimed loudly as he launched himself into a final assault against Selene.

With one decisive strike, he plunged his sword toward the heart of the dark spell, shattering the shadows. The cavern filled with brilliance, blinding light erupting from the orb, washing over Selene, and dispelling the darkness that had claimed many souls before her.

For the first time, the shadows retreated, yielding to the pure essence of valiant hearts. Selene screamed, the shadows swirling around her, merging with the light until both erupted in a stunning spectacle.

When the light faded, Cedric and his knights found themselves standing in the cavern, now serene and peaceful. The dark sorceress lay defeated, her power extinguished.

As they emerged from the cavern into the warm sunlight, they knew that the shadows had been vanquished—for now. The bond that had grown between Cedric and his knights solidified with the realization that they had faced their greatest fears together.

Return to Valoria was met with thunderous applause. The people embraced them wholeheartedly, grateful for the sacrifice and courage shown in the face of darkness.

Yet Cedric understood that the fight against despair would never truly end. He pledged to continue building the Order of the Light, instilling hope and resilience in every citizen.

As seasons passed, Valoria became a beacon of strength, an example to other kingdoms. Cedric tirelessly taught and mentored, telling tales of bravery and joint strength against the dark forces that lurked in forgotten corners.

And through it all, Lyra remained at his side, steadfast and fierce, embodying the spirit of the order. Together, they would face any challenge, holding on to the light that bound them, their bond as unyielding as the earth under their feet.

In Cedric's heart bloomed a lasting truth: as long as they stood united as one, darkness could never truly extinguish the light of hope.

Then, one fateful day as Cedric indulged in a leisurely stroll through the castle grounds, he caught sight of a radiant light flickering through the trees. He followed the glow, curiosity guiding him as he entered Eldwood, where hope had flourished.

At the heart of the forest stood Aurelia, the Guardian of the Eldwood, her aura shimmering like the night stars above. Recognizing Cedric’s presence, she smiled, warm and knowing. “You’ve done well, Sir Cedric. Valoria thrives under your care, but remember, the journey for peace requires constant vigilance.”

“What new challenges await us, Aurelia?” Cedric asked, pondering her wisdom.

“A new force stirs beyond the mountains, an ancient prophecy entwined with the energies of darkness and light. Seek it out when the time is ripe, for it may hold the key to lasting balance. Ensure that your courage and the bonds you forge remain steadfast,” Aurelia relayed, her words resonating with power.

Feeling the weight of her message, Cedric nodded solemnly. “Thank you, Aurelia. We will remain vigilant and ready for what lies ahead.”

As the sun dipped below the horizon, casting a warm glow over the kingdom, Cedric felt a renewed sense of purpose twine through him. Together, the Order of the Light would face the mysteries that awaited them beyond the horizon, transforming darkness into bonds of unity and hope.

And so, the legacy of Sir Cedric and Lady Lyra continued to flourish. The heartbeat of Valoria pulsed with promise, echoing tales of courage, love, and the unwavering light that would stand tall against the shadows for all time.

The flame of the Order would live on, unwavering against doubt, courage entwined in the marrow of every knight. They had seen darkness, but together, they had chosen to embrace the light. And in that choice, they left behind a legacy that would inspire both present and future generations to cherish hope, embrace love, and never waver in the face of fear.

With Cedric and Lyra leading the way, the dawn of a new era began, where the heart of Valoria remained unfaltering—a beacon shining brightly against the ever-looming shadows, forever warding off despair.

For every moment lived, they carried the tales of the past, lessons of bravery forever etched in their hearts, preparing to face the rising sun, relentless and glorious, illuminating the path ahead.

As the golden rays of dawn painted the skies over Valoria, a palpable sense of anticipation thrummed in the air. The kingdom had prospered since Cedric and Lyra's victory over Selene, the Shadow Weaver, and the establishment of the Order of the Light. However, Cedric could not shake the feeling that the time for complacency was over; the winds whispered of change, and in the depths of Eldwood, a prophecy was awakening.

The Order had flourished, drawing members from all walks of life eager to protect their homeland. Young recruits trained diligently, honing their skills in swordplay and archery while also learning the values of resilience, friendship, and bravery. Cedric and Lyra often led training sessions, guiding the recruits through exercises that tested both body and spirit.

One crisp morning after a particularly spirited training session with the younger knights, Cedric and Lyra gathered the group to discuss their next steps. “Today, we honor not only our training but also the bonds we forge here,” Cedric began, addressing the eager faces before him. “We will embark on a journey beyond Valoria’s borders to seek knowledge of the ancient prophecy Aurelia mentioned. It is time we learn more about what we may face.”

The faces of the young knights lit up with excitement and determination, the flicker of adventure igniting their spirits. Lyra stepped forward, her energy infectious. “This is an opportunity for growth! We’ll learn together and prepare for any challenges that await us. Who’s ready to stand with us on this quest?”

“I am!” a chorus of voices responded, and the excitement morphed into fervent cheers that resonated through the training grounds.

As they prepared for their journey, gathering supplies, armor, and weapons, Cedric found himself reflecting on the bonds they had cultivated. Each recruit was more than just a soldier; they were family, woven together by shared experiences, victories, and losses. It was this unity that bolstered his resolve to protect Valoria at all costs.

Among the recruits, a young man named Roland stood out. He possessed an infectious spirit and remarkable agility in training but lacked confidence in his abilities. Cedric had noticed this and recognized that he needed to be nurtured. “Roland,” Cedric called to him as they packaged their supplies. “You’ll be joining us on this journey. I’d like you to lead the scouting team. I believe in your potential.”

Roland’s eyes widened in surprise, disbelief taking hold. “Me? But what if I fail? What if I’m not good enough?”

Lyra stepped in, placing a reassuring hand on his shoulder. “You’ve trained with us, you’ve shown tenacity and courage, Roland. Remember, all of us have faced fear, but we move forward together. This is your chance to shine.”

With a hesitant nod, Roland accepted the challenge, determination beginning to flicker in his chest.

As the sun began to set, casting hues of orange and purple across the horizon, the knights gathered around Cedric and Lyra for their final briefing. “We leave at dawn,” Cedric declared. “Stay alert and trust one another. The path ahead will be filled with unknowns, but we will face them together.”

With spirits high and resolve hardened, the Order set forth at first light, leaving the familiarity of Valoria behind them. They rode through valleys adorned with wildflowers and crossed babbling brooks, their laughter echoing against the hills. The camaraderie was palpable, fortifying their determination as they ventured into uncharted territories.

Days passed as they journeyed farther from home. As they traversed forests and crossed wide rivers, the air grew denser with mystery and intangible whispers from the ancient lands. Each evening, they shared tales around the campfire, the bonds between them growing richer with every shared story and laugh.

One crisp evening, as they gathered to rest, Cedric noticed a peculiar tension in the air. The flickering flames cast an uneasy glow, and the shadows danced on the edges of their camp. He looked to Lyra, who seemed to sense it as well.

“Something feels amiss, doesn’t it?” Cedric asked, his voice low, barely above a whisper.

“I feel it too,” she replied, scanning the darkened trees lining their campsite. “We should keep watch tonight, just in case.”

The knights nodded, understanding the importance of vigilance. They set up two rotating watches as the moon climbed high into the sky, shrouding the world beneath a blanket of silver.

The night passed quietly until an eerie hush fell over the land. It was broken suddenly by the loud rustle of leaves. Cedric snapped awake, instincts kickstarting as he surged to his feet, summoning the knights to arms.

“Stay alert! There’s something in the trees!” he commanded, eyes scanning the darkness.

Before they could react, shadowy figures emerged from the forest, ethereal beings with glowing eyes, flickering in and out of sight. Whispers filled the air like a chilling wind, weaving a tapestry of fear.

“They’re back,” Lyra said, her face tense. “Those remnants of Malakar’s darkness!”

Cedric gripped his sword tightly and raised it high. “Knights! Stand together! Our light is brighter than any shadow!”

As the figures advanced, the knights organized themselves, forming a protective circle around their camp. Each knight brandished their weapons, rallying their spirits as they prepared to face the ethereal creatures.

Suddenly, a loud howl pierced the night, and the shadowy figures surged forward, melded with a dark magic that felt cold as ice. The air crackled with tension as the knights clashed with the shadows. Swords met shadow, the enemies coiling around them like a smoke cloud, seeking to extinguish the light.

Lyra fought fiercely, arrows flying with pinpoint accuracy, but the dark beings were relentless, swirling around her and trying to envelop her in despair. “Stay focused!” she shouted, directing her arrows toward the eyes of the nearest shadow.

Cedric charged into the fray, his sword glowing with the power of the orb. “Remember the light within! Focus your energy on the bond we share!” he implored, the force of unity empowering each knight around him.

As they fought, the knights drew from the well of their shared courage, the light emanating from their hearts shining brighter. With each defeat of a shadow, it dissolved in a cloud of shimmering dust, the whispers fading into the ether.

In the chaos of battle, Cedric caught sight of Roland, who, despite being engulfed in shadows, stood his ground. “You can do this, Roland! Remember your training!” Cedric encouraged him.

Gathering his resolve, Roland steadied himself, recalling the skills he had honed with Cedric and the others. With newfound courage, he thrust his sword into the heart of the shadow before him. “For Valoria!” he cried, dispelling the dark haze and breaking free of the grip of despair.

Soon, with their spirits ignited, the knights began to closer in and push back the forces of darkness. They became a united front, each swing of their swords punctuated by shouts of defiance.

The shadows, once powerful, began to retreat, struggling against the light emanating from the knights. With one final charge, Cedric and Lyra led the knights forward, combining their powers. “Together!” they shouted, thrusting their weapons forward in unison.

The blinding light erupted, filling the clearing, shining brighter than they had ever imagined. The shadows screamed as they were consumed by the brilliance, each whisper of despair faltering in the face of their unity.

As the light began to fade, Cedric lowered his sword, breathing heavily as the last wisps of shadow dissipated into nothingness. “We did it,” he gasped, looking at the weary but triumphant faces of his knights.

The battlefield stood silent now, the air clear as night reclaimed its serenity. They had faced the remnants of darkness and emerged victorious.

Roland stepped forward, a newfound strength in his stride. “Thank you for believing in me, Sir Cedric. I didn’t think I had it in me.”

Cedric smiled warmly at him. “You proved that courage is not the absence of fear but the resolve to face it. You were brave tonight, Roland. Together, we learn and grow.”

As dawn crept over the horizon, painting the sky in shades of rose and gold, the Order of the Light celebrated their victory while also reflecting on the dangers that still lurked in the unknown. They knew well that the remnants of Malakar’s dark influence were not entirely vanquished; they still felt the undercurrents of shadow weaving through the realm.

With each step toward their journey’s end, Cedric felt strengthened by the lessons learned and the camaraderie shared with his knights. They were not just fellow warriors but a family bonded through trials and triumphs that promised endurance in the face of future challenges.

As they resumed their travels, the group delved deeper into the heart of the lands, where rumors suggested the ancient prophecy lay waiting for a hero worthy of its secrets. Through valleys and over mountains, each stride brought them closer to uncovering the truth.

Days turned into weeks, and the journey grew ever more arduous. The landscapes shifted from lush forests to barren hills, and remnants of past civilizations caught their attention. Ruins crumbling under the weight of time whispered stories of ancient magic, sparking Cedric's curiosity.

On one such day, as they explored a forgotten temple, they stumbled upon murals adorned with vibrant, intricate designs depicting scenes of light battling darkness. One mural caught Cedric’s eye—a woman standing tall, encircled by the light, quelling monstrous shadows. He recognized her as a figure reminiscent of Aurelia.

“What if this is a depiction of our past battles?” Cedric mused aloud.

“I believe it is,” Lyra agreed, tracing the lines with her fingers. “It speaks to the power of unity between light and dark—how they are bound together, always at odds, yet integral to each other’s existence.”

Suddenly, the ground beneath them quaked, an ancient pulse echoing throughout the halls of the temple. Dust fell like rain from the ceiling, and Cedric steadied himself against a wall. “Everyone, stay alert!” he commanded.

The rumbling subsided, revealing a hidden passageway shimmering with golden light, beckoning them closer as if waiting for them to uncover its secrets. With a mix of trepidation and excitement, they entered the chamber.

At its center lay a magnificent pedestal, and upon it rested an ancient tome. Wisps of energy danced around it, pulsing with an arcane glow that compelled them to step closer. Cedric approached cautiously, feeling the weight of destiny pressing upon him.

“This tome could hold knowledge of the prophecy!” he exclaimed, knowing it might contain vital information for their journey.

As he reached out, Lyra cautioned, “Be careful! Such ancient knowledge may come with a price.”

Ignoring her warning, Cedric lifted the tome, its pages fluttering open as if welcoming him. The chamber filled with light, and images swirled before them—visions of light entangled in a fierce duality with shadow.

In a visual spectacle, Cedric saw the story of Valoria unfold—the rise of light, the birth of darkness, battles fought, creatures unleashed. The images crescendoed to a climax, revealing a prophecy warning of a time when darkness would rise again, and only those who embraced both light and shadow could restore balance.

The final vision centered on Cedric and Lyra, their figures entwined with the energy of the heart of Valoria, emanating warmth and unity, shining brighter than any star.

“That’s us!” Cedric said, filled with awe and a sense of purpose. “We are destined to protect the balance. We can’t allow darkness to claim our realm.”

Suddenly, the tome crackled with energy, and it spoke—its ancient voice echoing through the chamber: “Guardians of light and shadow, your destined path unfolds. Embrace the duality within to restore the fractured realms. The choice lies in understanding—the light thrives through acceptance, and the shadows can guide you.”

As the voice faded, the light dimmed, leaving them standing in solemn silence, digesting the weight of the knowledge imparted.

“We are more than just knights; we have become a part of something greater,” Lyra said, the truth settling in her heart.

With the revelation of the prophecy engraved in their minds, Cedric and his knights prepared to return to Valoria. They had so much to share, lessons learned and knowledge gained, but foremost was the understanding that their journey was not just about fighting against darkness but about embracing all aspects of existence.

Upon their return, the kingdom welcomed them back with open arms. Celebrations erupted in the streets, but Cedric felt the call to gather everyone. Summoning his knights, villagers, and leaders of the Order of the Light, he stood before them, the tome clasped in his hands.

“We have faced challenges and overcome darkness, but we’ve also learned that true strength lies in embracing all aspects of ourselves! The prophecy has revealed that we must unify our purpose,” he declared, gazing into the eager faces before him.

“There is much more to the world than mere light and dark; there is growth, understanding, and wisdom. We will continue to train and understand our duality, and through this, Valoria will emerge stronger than ever!”

The crowd roared with approval, excitement rippling through the air as hope ignited anew.

As the days turned into weeks, Valoria flourished, a blend of light and shadow weaving together to create a tapestry of resilience. Cedric and Lyra led the Order in training, combining lessons of strength and understanding, ensuring that each knight learned to face their fears, channel their powers, and honor the light and shadows inherent in themselves.

Together, they prepared for the unknown, embracing the best and worst of what lay ahead, always united, always shining brightly against the dark. And as they stood on the precipice of what was to come, they faced the horizon with strength and courage—a knight and a guardian, bound by hope and woven with the light of Valoria forever shining in their hearts.

Through all they had encountered, they had forged a legacy that would outlast them, echoing through time as a beacon for future generations, whispering of courage and unity, embracing the fluid dance of light and darkness. It was their journey to protect and nurture, assuring that Valoria thrived amidst the shadows that could never truly extinguish the light, forever forging onward.
""""Here is a longer story to summarize:\n"
    ),

    # 3rd prompt where the LLM's output should contain 1000 tokens
    (
        "Please generate a longer version of the following story in exactly 1000 tokens:\n"
        "In the land of Mystoria, there existed a magical forest known as Everwood. This forest was home to many fantastic creatures, including fairies, talking animals, and even gentle giants. "
        "For centuries, the people of Mystoria lived in harmony with the creatures of Everwood, respecting the balance of nature and the enchantments that kept the forest thriving. "
        "However, as time passed, a dark shadow began to loom over the land. A greedy sorceress named Morgath sought to harness the ancient magic of the forest to gain power and wealth. "
        "She devised a cunning plan to trap the forest's magic and cast a dark curse upon Mystoria. To do this, she enchanted a crystal that would draw magical energy from Everwood. "
        "The creatures of the forest sensed the impending danger and called upon a brave young girl named Lila, who had always felt a deep connection to nature. "
        "Lila, possessing a kind heart and a fierce spirit, agreed to help. She embarked on a journey to gather allies: a wise old owl named Orion, a courageous squirrel named Nibbles, and a gentle giant named Bramble. "
        "Together, they journeyed deep into Everwood, navigating enchanted paths and overcoming obstacles set by Morgath. Each had unique abilities that complemented one another, helping to unveil secrets and unlock the magic of Everwood. "
        "As they neared Morgath's lair, they encountered challenges that tested their unity and courage. Morgath's minions, formed from dark shadows, tried to thwart their progress, but with teamwork and clever strategies, Lila and her friends prevailed. "
        "Finally, they reached the sorceress's lair, where the crystal pulsed with dark energy. A fierce battle ensued as Lila confronted Morgath, who underestimated the strength of friendship and love for nature. "
        "In a climactic moment, Lila used her bond with the forest to channel pure magic, breaking the curse and freeing Everwood from Morgath's grasp. The sorceress was vanquished, and the land rejoiced as nature began to heal. "
        "As a token of gratitude, the creatures of Everwood gifted Lila with a pendant that shimmered with natural magic, ensuring that she would always remain connected to them. "
        "From that day on, Lila became the guardian of Everwood, teaching future generations the importance of protecting nature and living in harmony with its magic."
    ),

    # 4th prompt where the LLM's output should contain approximately 5000 tokens
    (
        "Generate a story that is having 5000 toknes"
    ),

    # 5th prompt for summarization
    (
        "Please summarize the following paragraph:\n"
        "Artificial intelligence has rapidly evolved in recent years, transforming various industries and daily life. From machine learning to natural language processing, AI technologies have allowed for improved efficiency, smarter decision-making, and enhanced customer experiences. Companies are leveraging AI to analyze vast amounts of data and derive insights that were previously unattainable. As AI continues to advance, ethical considerations become increasingly important, prompting discussions around the implications of AI systems in society. It is crucial to balance innovation with responsibility to ensure that the benefits of AI are realized while minimizing potential risks and biases."
    )

]

# Run the evaluation
evaluation_results = evaluate_models_with_prompts(prompts_to_test)

# Print the results
for result in evaluation_results:
    print(f"Prompt: {result['prompt'][:60]}... (Input Characters: {result['input_characters']}, Input Tokens: {result['input_tokens']})")
    print(f"Azure OpenAI - Time: {result['azure_time']:.2f}s, Characters generated: {result['azure_characters']}, Tokens generated: {result['azure_tokens']}")
    print(f"Llama 3 (AWS) - Time: {result['llama_time']:.2f}s, Characters generated: {result['llama_characters']}, Tokens generated: {result['llama_tokens']}")
    print("-" * 50)

Prompt: Summarize the following story - In the heart of the enchante... (Input Characters: 8834, Input Tokens: 2209)
Azure OpenAI - Time: 1.89s, Characters generated: 1032, Tokens generated: 258
Llama 3 (AWS) - Time: 22.91s, Characters generated: 10117, Tokens generated: 2530
--------------------------------------------------
Prompt: Summarize the following story - In the faraway kingdom of Va... (Input Characters: 48829, Input Tokens: 12208)
Azure OpenAI - Time: 3.83s, Characters generated: 2313, Tokens generated: 579
Llama 3 (AWS) - Time: 27.14s, Characters generated: 10121, Tokens generated: 2531
--------------------------------------------------
Prompt: Please generate a longer version of the following story in e... (Input Characters: 2377, Input Tokens: 595)
Azure OpenAI - Time: 16.26s, Characters generated: 4598, Tokens generated: 1150
Llama 3 (AWS) - Time: 22.27s, Characters generated: 10355, Tokens generated: 2589
--------------------------------------------------
Prompt: Gener

## Removing the max limit from the LLM settings

In [ ]:
def call_llama3_1_aws_nomax(prompt):
  aws_region = ''
  access_key_id = ''
  secret_access_key = ''

  # Create a Bedrock client
  bedrock = boto3.client(
    'bedrock-runtime',
    region_name=aws_region,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key
  )

  model_id = ""
  native_request = {
    "prompt": prompt,
  }

  request = json.dumps(native_request)
  try:
    response = bedrock.invoke_model(modelId=model_id, body=request)
    response_body = json.loads(response.get('body').read())

  except ClientError as e:
    # Handle any errors that occur during the invocation
    print(f"An error occurred: {e.response['Error']['Message']}")

  return response_body.get('generation')

In [ ]:
def call_azure_openai_nomax(prompt):
    client_openai = AzureOpenAI(
        api_key="",
        api_version="",
        azure_endpoint=""
    )

    response = client_openai.chat.completions.create(
        model="",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

  #max input = 128,000
  #max output = 16,384

In [ ]:
def count_tokens(text):
    # Approximation: 1 token ~ 4 characters, adjust as necessary
    return len(text) // 4 + (len(text) % 4 > 0)  # Round up for any remaining characters

def evaluate_models_with_prompts(prompts):
    results = []

    for prompt in prompts:
        # Measure Azure OpenAI
        start_time = time.time()
        azure_output = call_azure_openai_nomax(prompt)
        azure_time = time.time() - start_time
        azure_characters = len(azure_output)
        azure_tokens = count_tokens(azure_output)  # Count tokens in the output

        # Measure Llama 3 on AWS
        start_time = time.time()
        llama_output = call_llama3_1_aws_nomax(prompt)
        llama_time = time.time() - start_time
        llama_characters = len(llama_output) if llama_output else 0
        llama_tokens = count_tokens(llama_output)  # Count tokens in the output

        # Store results
        results.append({
            'prompt': prompt,
            'input_characters': len(prompt),  # Count characters in the prompt
            'input_tokens': count_tokens(prompt), # Count tokens in the prompt
            'azure_time': azure_time,
            'azure_characters': azure_characters,
            'azure_tokens': azure_tokens,
            'llama_time': llama_time,
            'llama_characters': llama_characters,
            'llama_tokens': llama_tokens,
        })

    return results

In [ ]:
# Example prompts for testing
prompts_to_test = [
    # 1st prompt with approximately 1000 input tokens
    (
        """Summarize the following story - In the heart of the enchanted kingdom of Eldoria, there lay a village called Oakshade. This village was known for its lush, green trees and sparkling rivers, which glimmered like diamonds in the sunlight. However, unbeknownst to its inhabitants, a mysterious darkness was beginning to envelop their land.

One sunny morning, a young woman named Elara woke up with a sense of unease. Ever since she had heard the whispers of the townsfolk about a Shimmering Stone hidden deep within the Forest of Shadows, she couldn’t shake the feeling that destiny was calling her. The Shimmering Stone was said to hold immense power, capable of either saving the kingdom or bringing about its downfall, depending on who wielded it.

Elara had always been adventurous, often exploring the edges of the Forest of Shadows that bordered their village. Despite the dark rumors that surrounded it, she had collected beautiful flowers and rare herbs from the forest to help her family and neighbors. But now, a heavy heart weighed on her as she recalled stories from the elders about the stone’s significance. They spoke of its glow, which once filled the entire kingdom with light and warmth. Elara felt compelled to uncover the truth.

Determined, she packed a small satchel with supplies: dried bread, a waterskin, her trusty dagger, and a worn map handed down from her grandmother. The map promised to lead her to the heart of the forest, where the Shimmering Stone lay hidden. Just as she was about to leave, her younger brother, Finn, rushed to her side, his blue eyes filled with excitement.

“Let me come with you, Elara! I can be brave too!” Finn pleaded, bouncing on the balls of his feet.

Elara hesitated. The journey would be perilous, but she couldn’t ignore the fire of determination in her brother’s eyes. After giving it a moment's thought, she knelt down and said, “Alright, but we must stick together and promise to return home.”

With a shared sense of purpose, the siblings ventured into the Forest of Shadows. The canopy overhead loomed dark and forbidding, with twisted branches that seemed to whisper secrets as they brushed against one another. As they walked deeper into the forest, the sunlight faded, replaced by a cool, eerie twilight.

After several hours of walking, they reached a clearing filled with shimmering blue flowers that seemed to glow. In the center of this natural oasis stood a tall fountain with crystal-clear water flowing from it. Elara remembered the tales of the Fountain of Reflection, said to reveal one’s true feelings when gazed upon.

“Should we look?” Finn asked, curiosity piqued.

Elara nodded and leaned over the fountain’s edge, gazing into the water. Instead of her reflection, she saw herself standing before the Shimmering Stone, its light illuminating a path forward. But the image quickly shifted, revealing shadows chasing her and Finn, darkness closing in.

“I... I don’t think we should look too long,” Elara said, pulling Finn away. The siblings breathed heavily, both feeling the weight of what they had seen.

Suddenly, a rustling sound startled them, and a figure emerged from the bushes. It was an old woman with a crooked back and a long, flowing robe made from leaves and vines.

“Seekers of the Shimmering Stone, why do you trespass in these woods?” she croaked, her voice both intimidating and gentle.

“We seek the Shimmering Stone to save our village from darkness,” Elara replied bravely, her heart racing. “Can you help us?”

The old woman studied them for a moment, her gray eyes sparkling with intrigue. “Many have sought the stone, but only those with pure hearts may claim its power. You will face challenges that will test your courage and bond as siblings. Only then will the stone reveal itself.”

With a wave of her hand, the old woman conjured a map of the forest, illustrating all the hazards ahead. “Beware of the shadow creatures that await. They feed on fear and doubt. Trust in each other, and remember, the light of hope will guide your way!”

Thanking the woman, the siblings pressed forward, clutching the map tightly. Soon, they encountered their first challenge: a vast chasm that seemed to stretch infinitely below. The only way across was a series of narrow, swaying bridges suspended high above the abyss.

“Do you think we can cross these?” Finn asked, his voice shaking.

“I know we can,” Elara encouraged. “Just hold my hand and follow my lead.” Taking a deep breath, they grasped each other’s hands and stepped onto the first bridge. The wooden planks creaked dangerously, and Finn’s knees felt weak. Panting, he looked at Elara, who smiled reassuringly.

“Keep going,” she said. “Just focus on me.”

With each step, they found their rhythm. The sway of the bridge felt less daunting with their combined strength. Finally, they reached the other side, where they collapsed in a mix of relief and laughter.

“See? We did it!” Elara exclaimed.

But their joy was short-lived. As they continued on their path, the air thickened with an overwhelming darkness, and shadow creatures emerged from the trees. They were ghostly figures, their forms swirling and flickering like smoke.

“Listen to the whispers of doubt,” the shadows hissed. “You’ll never find the stone. You’re just weak children!”

Finn trembled, fear gripping his heart. “What do we do, Elara?”

“Remember what the old woman said! They feed on fear,” Elara whispered fiercely, trying to suppress her own rising anxiety. “We have to stay strong and believe in each other.”

Drawing strength from her brother, she faced the shadows. “You’re wrong! We have the courage and love that no darkness can extinguish!” she shouted, her voice rising above the whispers.

Unexpectedly, a brilliant light erupted from their joined hands, pushing back the shadows. The creatures shrieked, recoiling as if struck by a powerful force. The siblings tightened their grip, feeling the warmth of hope envelop them.

With the shadows dissipated, they pressed on, energized by this victory. Hours later, they finally approached an ancient stone temple flanked by two massive statues of guardians. The air shimmered with magic, and in the temple’s center, they saw it—the Shimmering Stone, glowing like a thousand stars.

“It’s beautiful!” Finn gasped, his eyes wide with wonder.

As they stepped closer, the stone pulsed with energy, illuminating the room. But before they could touch it, a voice echoed through the temple. “Only those who truly understand the meaning of sacrifice may claim the stone.”

Elara and Finn looked at each other, puzzled but determined. “What do we need to sacrifice?” Elara asked.

“Prove your love for one another,” the voice continued. “Only by placing what you cherish most on the altar may the stone be yours.”

Elara’s heart sank. Both she and Finn were aware of their deep bond, but this was a test unlike any other. Finn hesitated, clutching his satchel, filled with a few cherished trinkets from their childhood. Meanwhile, Elara held onto a locket of their late mother—something she had always kept close to her heart.

“I can’t give up Mom’s locket!” Elara cried. “It’s all I have left of her!”

Finn nodded, understanding the weight of her words. “And I can’t part with my toys, Elara. They remind me of the good times we had!”

“Maybe…” Elara’s voice trailed off. “Maybe it’s not about giving up our most cherished things, but understanding the value of love and courage we share.”

Taking a deep breath, Finn stepped forward, placing his satchel on the altar. “I would trade my favorite things any day for our family’s safety.”

Moved by his selflessness, Elara followed suit, placing her locket beside his satchel. They both stepped back, watching as the stone blazed brightly, lighting up the entire temple. With a hum of energy, the stone floated closer, enveloping them in a warm, golden light.

In that moment, they knew they had passed the test. The light faded, and the Shimmering Stone glided down to them, resting softly in their hands. It shone radiantly, as if the warmth of love had been infused into its very essence.

“You have proven yourselves to be worthy,” the temple echoed. “Together, you shall harness the stone’s power to heal your kingdom.”

Armed with the stone, Elara and Finn exited the temple, their hearts filled with hope. The darkness that loomed over Oakshade had vanished, replaced by the brilliant light radiated by the Shimmering Stone.

As they returned home, the villagers gathered around, amazed at their bravery. With the Stone’s power, Elara and Finn restored light and prosperity to their homeland, their bond now stronger than ever.

From that day forward, the tale of Elara and Finn became legend in Eldoria, a reminder that courage, love, and sacrifice could conquer the deepest darkness. "
"""
    ),

    # 2nd prompt with approximately 10,000 input tokens
    (
        """Summarize the following story - In the faraway kingdom of Valoria, where the sun kissed the earth with golden rays and the hills danced with wildflowers, there lived a legendary knight named Sir Cedric. His deeds were sung by bards in every tavern, and children idolized him as a hero, but few knew the true weight of his responsibilities. Sir Cedric was not only known for his bravery but also for the kindness that flowed from his heart like the rivers that nourished the land.

One day, as Cedric gazed out from his castle window, a sense of foreboding nestled in his chest. The horizon, usually vibrant with life, appeared dimmer. The villagers had been unusually quiet, their laughter replaced by whispers filled with worry. Without delay, Cedric donned his armor, polished and gleaming as bright as the morning sun. It was time to uncover the source of this unease.

Riding through the village, Cedric was greeted by filled teeming streets lined with colorful stalls. The scent of bread wafted through the air, mingling with the earthy aroma of fresh hay. Yet, the joy that once filled the hearts of the villagers seemed stifled, weighed down by an invisible force. As he approached the central square, he noticed old Mrs. Hargrove sitting by her stall, her hands trembling as she knitted.

“Good day, Mrs. Hargrove,” Cedric called cheerily. “What troubles you on this fine morning?”

She looked up at him with weary eyes. “Oh, Sir Cedric, the crops have failed once again, the livestock are restless, and the river’s flow has dwindled! We fear there may be dark forces at work, and people are becoming desperate.”

Cedric's heart sank. The land of Valoria thrived on its crops and rivers, and he knew desperation could drive even the gentlest soul to madness. “Fear not, good lady. I will investigate this matter and bring back prosperity for our home,” he assured her, determination burning in his chest.

As he rode toward the Eldwood Forest, the shadows grew longer, and a dense fog began to loom. Eldwood was both beautiful and treacherous, known for its towering trees and sparkling rivers but also for the legends that spoke of ancient magic and creatures that roamed its depths. The further Cedric ventured, the more he felt a presence, as if the trees themselves were watching him with bated breath.

Passing through the forest's edge, Cedric's keen eyes caught sight of a flickering light in the distance. Curiosity piqued, he ventured closer. As he approached, he stumbled upon a clearing where a brilliant azure flame danced upon a stone pedestal. Atop it rested a crystal orb, shimmering with an otherworldly radiance.

“Mystical enchantment,” he murmured, reaching out towards it, but a voice stopped him in his tracks.

“Only those pure of heart may take from the Keeper of the Flame,” the voice echoed, resonating through the clearing. From the shadows emerged a figure – a woman draped in robes that glimmered like stars, her hair flowing like rivers of light.

“I am Aurelia, Guardian of the Eldwood,” she proclaimed. “Many have sought my orb, believing it to contain the answers to their woes. But its power comes with a price.”

“What price?” Cedric inquired, his mind racing. He needed to understand if this orb held the salvation his people sought.

“To wield the orb, you must confront the truths within your soul, face the shadows of your past, and emerge whole. Only then can it grant you the strength to combat the forces that plague Valoria,” Aurelia explained, her eyes piercing yet warm.

Noble and brave as he was, the notion of introspection filled Cedric with doubt. He had spent years combating foes in physical battles, but the battle of the soul was one he had never faced. Still, he understood the consequences of inaction and voiced his intent. “I will confront whatever lies within. I owe it to my people.”

“Very well,” Aurelia said, her voice steady. “Approach the orb and lay your hand upon it.”

Cedric stepped forward, heart pounding, and placed his hand on the cool surface of the orb. Suddenly, a surge of energy coursed through him, blinding him in a flash of light. When he opened his eyes, he found himself standing in a long corridor dimly lit by flickering torches. The air was heavy, filled with the whispers of the past.

“Welcome to the Realm of Reflection,” the voice of Aurelia echoed through the corridor. “Here, you shall face your memories, your regrets, your greatest fears.”

Without warning, scenes began unfolding around him. The first memory to emerge was from his childhood: a young Cedric stood before a large wooden door, the sounds of laughter echoing behind it. His father was hosting a grand feast, but Cedric, shy and uncertain, refrained from joining the celebration.

“Why didn’t you ever join them?” Aurelia’s voice was now a soft whisper, coaxing him to confront his choices.

“I felt like an outsider,” he replied, struggling with the emotions from that long-forgotten day. “I was scared of not being accepted.”

The memory shifted, showing him the day he was knighted. Cedric stood proudly in shining armor, but his eyes betrayed the burden of expectations resting on his shoulders. He saw his mentors smiling, the townspeople cheering, yet he felt an irrepressible fear – the fear of failing them, of not living up to the title of knight.

“Your fears are a part of you, Sir Cedric,” Aurelia said. “But they do not define you. You must understand that true courage lies within acceptance, not just of yourself but also of the vulnerability that comes with it.”

With each passing memory, Cedric faced moments of doubt, anger, and grief. He relived significant failures—the times he had let his friends or fellow knights down. Each memory warped into a haunting specter, whispering words of defeat, urging him to surrender.

But through the darkness, a flicker of light emerged with each confrontation. With each moment faced, he felt a weight lift as he embraced his flaws, realizing they were not signs of weakness but colors on the canvas of his life.

Finally, he found himself standing before a great mirror, its surface dark and murky. “To finish this journey, you must look into the mirror and accept who you are,” Aurelia guided him.

Taking a deep breath, Cedric stepped forward. As he gazed into the depths of the mirror, his reflection transformed. He saw himself not only as a knight but as a man, vulnerable yet strong, filled with untold scars but also untapped potential. The mirror shimmered, encapsulating his past, present, and future in a whirlwind of light.

“I am Sir Cedric, a knight of Valoria. I embrace my past and the fears that once confined me,” he declared, voice steady. “I accept my flaws and care for those I protect. I fight not only for honor but for love and hope. My journey has shaped me.”

At that moment, the mirror exploded into a cascade of colors, enveloping him in a warm glow. He felt renewed, empowered, and utterly freed. The realm began to dissolve around him, and with one final surge of energy, Cedric found himself back in the clearing, the orb still glowing on the pedestal before him.

“Congratulations,” Aurelia smiled softly. “You have faced your shadows and emerged stronger. You now hold the strength to protect Valoria.”

With newfound determination, Cedric lifted the orb and felt its power resonate through him. “Thank you, Aurelia. I will use this gift wisely,” he vowed.

With a gentle nod, Aurelia stepped back into the shadows of the Eldwood, leaving Cedric to navigate his way back to his village. As he returned, the skies began to clear, and the sun's warmth enveloped him, bringing back life to the flowers and trees.

The villagers marveled at his return, noticing the light emanating from him as if he had absorbed the essence of the sun. Cedric gathered them in the square, feeling the weight of their hopeful gazes upon him.

“My friends! I have faced a great trial and discovered the true strength that lies within us all!” he called out, the orb glowing beside him. “Together, we can overcome the darkness that has crept into our land. This orb will help us restore balance—the crops will flourish again, and peace shall reign!”

As Cedric spoke, the orb pulsed, sending waves of radiant light across Valoria. Crops began to bloom, the river’s flow surged, and laughter erupted once more among the villagers. Hope returned, stronger than ever.

Under Cedric’s vigilant watch, he led initiatives to support and uplift the villagers. They started a harvest festival, where everyone came together to celebrate not just the bounties of the land but the unity they had fostered through their struggles. The air was filled with music, laughter, and camaraderie as old wounds healed and new friendships blossomed.

Cedric often found children gathering around him, eager to hear tales of his adventures, but this time, he did not only recount battles but also spoke of vulnerability, courage, and the true essence of bravery that lay in accepting oneself.

As seasons turned, the kingdom flourished, yet Cedric’s desire to protect Valoria never waned. He knew whispers of darkness still roamed the far corners of the kingdom. So, he established a council, training young knights not only in combat but in wisdom and leadership.

Together, they spent hours discussing the meaning of true strength and the importance of embracing their fears. Cedric’s favorite was a young girl named Lyra, who possessed an uncanny talent for archery. She dreamt of becoming a knight and would often shadow Cedric, soaking in every lesson he shared.

One evening, as they practiced their skills, Lyra confessed, “Sometimes, Sir Cedric, I doubt myself. I’m afraid I’ll never be as good as you or the others.”

Cedric paused, seeing the same hesitation in her eyes that he once harbored. “Lyra, the journey to strength is not always about being the best. It’s about facing your doubts and nurturing your passion. Embrace who you are and cherish your journey. You are capable beyond your belief.”

In time, Lyra became one of Cedric’s most trusted knights, embodying the spirit he had instilled in her. They fought alongside each other, not just against external threats but for the wellbeing of those they loved.

But change often comes with trials. Word began to spread of a rising shadow from the North, a dark sorcerer named Malakar who thrived on chaos and despair. It was said that he sought to harness the ancient magic buried deep within Eldwood, forcing the unkind energies of despair over those who opposed him.

Determined to protect Valoria, Cedric gathered his knights and ventured into the distant lands to confront Malakar. The journey was fraught with peril, as they traversed mountains, crossed fierce rivers, and battled relentless storms. Each night around the campfire, Cedric reminded his comrades of their purpose—the love they shared for their land and each other.

Finally, they arrived at the fortress where Malakar resided, shrouded in a perpetual storm. The laughter of Valoria seemed a distant memory amidst the oppressive darkness. Cedric and his knights stood firm, ready to face what was to come.

They stormed the gates, swords gleaming in the faint light, undaunted by the specters that wailed in torment around them. Within the fortress, Malakar awaited, cloaked in shadows, his presence chilling the very air.

“Brave knights,” he mocked. “What brings you to my domain? Do you seek glory? Or do you simply wish to meet your end?”

Cedric stepped forward, unwavering. “We bring hope! Your darkness will not prevail over the light that bonds Valoria!”

With a wave of his hand, Malakar unleashed a wave of dark energy, sending Cedric and the knights sprawling. The shadows twisted and writhed, threatening to engulf them. But Cedric stood, fueled by the love and hope of his people.

Calling forth the orb’s power, Cedric raised it high, its azure flame igniting the battlefield. Surrounding him, his knights rallied, pushing against the dark tide. Lyra, by his side, loosed her arrows that glimmered like stars, striking the shadows with precision.

Together, they advanced towards Malakar, the orb illuminating the darkened fortress. The sorcerer, realizing his impending defeat, unleashed waves of despair, flooding the air with nightmares.

But Cedric remembered Aurelia’s words—the key was unity. Amidst the chaos, he raised his voice. “Remember the light within! Together, we are stronger than his darkness!”

Lyra ran to Cedric’s side, her courage shining as bright as the orb. The other knights joined, forming a circle of unity around Cedric. They focused their hearts and souls into the orb, channeling their fears and hopes, causing it to burst into blinding light.

The darkness faced a relentless tide, pushing back the shadows. With a powerful surge, they connected their spirits, empowered by the love that resided in their hearts. The orb erupted, sending beams of light cascading throughout the fortress, obliterating Malakar’s malevolence.

Screaming in fury, Malakar found his power waning, shadows seeping away. Realizing he had lost, he attempted to retreat, but it was too late. The light engulfed him, and with a final cry, he vanished into the ether, the fortress crumbling around them.

As silence fell over the fortress, Cedric and his knights emerged victorious. They had not only defeated the darkness but had illuminated a path of hope and belonging that would resonate across Valoria.

Returning home, they were met with joyous celebrations. The people of Valoria rejoiced, grateful for Cedric and his knights. But Cedric knew it was not just him—it was the unity, love, and acceptance they had nurtured.

In the many years that followed, Cedric continued to serve as a guiding light for his people. He ensured the teachings of courage, vulnerability, and unity were carried forward. The stories of their triumph over darkness turned into legends, inspiring generations to come.

His heart swelled with pride each time he recounted his journey alongside loyal knights like Lyra, and they forged a new legacy together where darkness could never extinguish the vibrant light of Valoria.

And so, the kingdom flourished under Cedric’s watchful eye, enveloped in peace and prosperity. Cedric never forgot Aurelia’s wisdom. He embraced every moment, knowing that true bravery lay not in the absence of fear, but in the courage to face it, hand in hand with those they loved.

In the quiet evenings, under the watchful stars, Cedric knew that a hero was not merely one who wielded a sword, but one who nurtured the light within every heart. And as the people of Valoria looked upon him with reverence, a new legend began—a legend not of a knight, but of a man who loved fiercely, dreamed boldly, and conquered darkness with the radiant power of hope.
As the celebrations in Valoria thrived, peace reigned for many moons, yet Cedric knew that the echoes of Malakar’s darkness still lingered in the realms beyond their borders. Though the kingdom flourished, he felt a whisper within him that the threat was not entirely vanquished. Strange rumors started creeping into the village: sightings of shadowy figures moving under the cover of night and villagers speaking of nightmares that seemed more vivid than before.

It wasn’t long before a council was convened to discuss the emerging unease. Cedric, accompanied by his trusted knights, seated himself at the round table in the grand hall of Eldoria Castle. Lyra sat beside him, exuding confidence and fire, her eyes reflecting her unwavering determination.

As the council gathered, Elder Garrick, the wisest among them, stood to address the assembly. “Our kingdom faces shadows anew. We must not allow complacency to dull our senses. The essence of Malakar might still corrode the edges of our peace.”

Cedric nodded gravely. “Elder Garrick speaks truthfully. If remnants of Malakar's power linger, we must seek them out, lest they fester into something much worse. I propose we dispatch scouting parties to investigate these claims and gather intelligence,” he suggested.

Lyra raised her hand, her expression fierce. “I wish to lead one of the scout parties, Sir Cedric. I’ve trained hard, and I believe I can uncover the truth of these shadows!”

“Bravery shines bright within you, Lyra. However, I must consider the dangers,” Cedric replied, appreciating her fervor.

“Cedric,” she insisted, “we’ve faced darkness before. I can handle this! I will not be a damsel in distress; I have proven myself time and time again. Trust me!”

The knight smiled, feeling pride swell in his chest. “Very well. I trust your instincts. Gather volunteers, and prepare a team. We shall move by dawn.”

As the council dispersed, Cedric remained deep in thought. He could feel shadows gathering on the horizon. He resolved to join Lyra on her scouting mission. Together, they would face whatever challenges awaited them, supporting each other as they always had.

The sun rose over Valoria, painting the sky with hues of orange and pink. As the knights assembled, Cedric and Lyra led the charge out of Eldoria Castle, their hearts united with purpose. The group rode through the fields and into the woods, where the shadows thickened, and a chill settled in the air.

The group’s journey was cautious but composed. They navigated through the dense trees and underbrush, keenly aware of their surroundings. After hours of riding, they reached a hidden clearing where they found remnants of a gathering. The ground was scorched, charred markings sprawling across the earth, reminiscent of dark magic.

Lyra knelt to examine the ground. “This was recently conjured. The darkness has a lingering essence—something evil transpired here,” she said, a frown creasing her brow.

As they continued their investigation, the sounds of distant murmurs permeated the air. With a gesture, Cedric signaled the knights to silence. They remained hidden behind the trees as a group of figures came into view, clad in ragged robes infused with dark energy. Cedric recognized their insignia—a shadowy emblem representing Malakar.

“Gather, my minions!” a figure commanded, shrouded in darkness. “We will harness the fears they leave unguarded, and the remnants of our master will grant us the power to rule Valoria once more!”

Cedric's heart raced as anger boiled within him. “These are remnants of Malakar's followers. They must not be allowed to gather strength!” he whispered urgently.

Lyra nodded, determination burning in her eyes. “We have to strike before they can amplify their dark energy. We cannot allow this to fester!”

Cedric formulated a battle plan. “We’ll create a distraction. My knights will provide cover while you and I flank from the east. We will catch them off guard!”

The knights mobilized quickly. With a battle cry, they charged, drawing the attention of Malakar’s followers. As the shadowy figures turned, Cedric and Lyra slipped around the flank, darting into the chaos.

“Remember our training!” Cedric yelled to the knights as they engaged the enemies. The battle was fierce; shadows clashed with shining steel, and the air crackled with magic.

As Cedric confronted the leader of the dark followers, his sword gleamed in the smoky air. “You shall not use this land for your darkness!” he declared, charging forward.

The figure cackled, summoning dark tendrils from the ground that lashed out at Cedric. He ducked and rolled, narrowly avoiding the attack. “Foolish knight! You think you can take on the shadows that serve a master?”

“Your master is defeated! The light will always overcome the dark!” Cedric shouted, channeling strength from within as he surged forward, sword striking true.

Lyra fought alongside him, her arrows striking with precision, illuminating the battlefield with their brilliance. As their enemies fell one by one, the tide of battle turned in favor of the knights. The shadows began to retreat, fear written across their darkened faces.

But the leader of the followers remained resolute. “You think you have won? You will never know true despair!” he growled, drawing forth an amulet that glowed ominously.

At that moment, a powerful wave of dark magic exploded from the amulet, blasting Cedric and Lyra across the clearing. The force knocked them to the ground, leaving them dazed and disoriented.

Cedric struggled to his feet, panic rising as he saw the shadows converging again. They were regrouping, preparing for one last attack. “We must not falter!” he urged Lyra, adrenaline pumping.

“I’m with you,” she replied fiercely, her resolve crystallizing as she stood by his side.

Using the strength of their bond, they charged at the remaining shadows together, cutting through their ranks, their blades flashing in unison.

Amidst the chaos, Cedric noticed that the figure was chanting—a ritual. Realizing what was happening, he shouted, “Lyra, stop him! We must break his focus!”

Lyra nodded, drawing a deep breath as she nocked an arrow to her bow. With a steady hand and clear mind, she aimed and released, the arrow flying true. It struck the amulet, shattering the dark energy it had been channeling.

With a blinding flash, the magic exploded, enveloping the clearing in light. Cedric shielded his eyes, feeling the warmth wash over him as shadows dissipated into the ether. When the light faded, he saw the leader of the shadowy figures fall to the ground, defeated.

The remaining shadows fled into the forest, scattering like leaves in the wind. Victory was theirs, but the battle was not without cost; the darkness had left scars upon the landscape.

As Cedric and Lyra caught their breath, their knights surrounded them, praising their leadership and courageous hearts. “We did it!” one knight exclaimed, holding his sword high.

“Victory belongs to all of you,” Cedric said humbly, grateful for his comrades. “But we must uncover more of this threat. The darkness may return.”

Lyra glanced at the remnants of the amulet, flickering with residual magic. “We should take this back to Eldoria. Perhaps we can learn more about it and find a way to prevent this from happening again.”

The group began their journey back, weary yet triumphant. Upon their return to Eldoria Castle, they were greeted as heroes. The people gathered to hear tales of their bravery, their spirits reinforcing the bonds that held the kingdom together.

Gathering once more in the council chamber, Cedric laid the shattered amulet on the table. “This dark magic could pose a threat again. We cannot afford to let our guard down.”

Elder Garrick leaned forward, studying the fragments. “Ancient magic such as this can be difficult to eradicate. It thrives on fear and despair, and we must remain vigilant in our teachings to fortify our people’s hearts.”

With that, Cedric, Lyra, and the other knights devoted their efforts to educating the citizens of Valoria. They implemented training programs, teaching self-defense and resilience against dark influences while fostering a greater sense of community and unity.

Days turned into weeks, and hope began to flourish once more across the land. The shadows of the past began to lift, and the people, emboldened by their knightly protectors, stood stronger together.

However, amidst the renewed spirit, Cedric felt the weight of vigilance tugging at his heart. He often found himself gazing into the horizon, aware that darkness never truly vanished; it merely lay in wait.

One night, as Cedric prepared for bed, a knock echoed on his chamber door. It was Lyra, her eyes radiant with excitement.

“Cedric! I’ve come up with an idea,” she said, breathing heavily. “With the training programs yielding great results, I believe we can establish an Order of the Light, where we can gather knights and civilians alike to guard against any resurgence of dark magic!”

“An Order of the Light?” Cedric mused, intrigued. “What would it entail?”

“Imagine! A brotherhood and sisterhood bound by honor and courage, fostering hope and knowledge. We could train together, share wisdom, and rally against the loom of darkness! Our unity would be our greatest weapon!” she exclaimed passionately.

Inspired by her vision, Cedric nodded earnestly. “That is a brilliant idea, Lyra. We will need to hold a council to discuss this further.”

The very next day, assembling their most trusted knights and the villagers, Cedric and Lyra presented the concept of the Order of the Light. The response was overwhelming; excitement filled the air as people rallied around the prospect of joining forces for a unified cause.

Iterations of training began, and as months passed, the Order of the Light grew, welcoming all who sought to protect their homeland. Cedric and Lyra stood at the forefront, mentoring and inspiring others to face their fears and cultivate inner strength.

As the Order established itself, they created ceremonies to honor the virtues of bravery, unity, and resilience. Together, they held festivals celebrating the light within, invigorating the spirit of Valoria and solidifying their bonds.

But as the Order flourished, rumors began to circulate about the dark remnants that still lingered in the world. Across distant lands, whispers of a new dark sorceress, one who coveted the power of shadows, began to surface. The air grew thick with unease again, and the council knew they could not remain idle.

Cedric gathered his knights once more, preparing them for the daunting road ahead. “The shadows are shifting anew,” he stated gravely. “We must venture beyond our borders and confront this threat before it reaches our kingdom.”

Lyra stepped forward, driving her fist into her palm. “Let us not forget what we’ve accomplished; our strength is forged in unity! Together, we can face whatever shadows arise!”

Determined, Cedric prepared the horsemen, arming them with knowledge, courage, and new spells learned from the remnants of Malakar’s dark magic that were now theirs to wield wisely. He felt a renewed energy pulsate within him, ready to stand against whatever may come.

They set out at first light, crossing mountains, traversing rivers, and journeying over grasslands. Each knight felt a sense of purpose, honor, and pride, driven forth by their commitment to protect Valoria.

After several days of travel, they approached a valley overshadowed by looming peaks. The air felt heavy, and a chill crawled up Cedric’s spine. They continued onward, determination bolstering each step until they found a hidden entrance to a cavern shrouded in dark mist. This was the rumored lair of the dark sorceress.

“Stay alert,” Cedric commanded, his heart racing as they crossed the threshold into the darkness. The group moved through the winding tunnels, lit only by flickering torches that cast eerie shadows against the stone walls.

Deep within the cavern, they found her—a striking figure draped in flowing black garments that shimmered like the night sky. A crown of obsidian crystals adorned her head, reflecting the dim light and emitting a low hum. Her presence exuded power, a sense of menace that thickened the air.

“Welcome, knights of Valoria,” she said, her voice melodic yet chilling. “I am Selene, the Shadow Weaver. You’ve come seeking to reclaim your precious light, but you will find only despair!”

Cedric stepped forward, undeterred. “We shall not allow you to weave your dark magic into our lands! We stand united to protect the light that binds us!”

“Such sentiment!” Selene laughed, the sound echoing off the cave walls. “You cannot comprehend the beauty of the shadows. They wield power unimagined. Embrace the darkness, and you may yet survive!”

With a wave of her hand, shadows coiled like serpents, striking at Cedric and his knights. In response, they formed a protective circle, swords drawn, ancient spells surging as they fought against the encroaching darkness.

“Remember the light within you!” Cedric shouted, just as he felt the familiar warmth of hope envelop him. The orb from Eldwood responded, igniting with radiance, illuminating the cavern with a brilliant glow.

Selene recoiled, disbelief washing over her. “No! How dare you wield such power!” she exclaimed as she conjured a dark spell, swirling shadows spiraling into fierce winds.

The knights pressed forward, their swords glinting in the light as they cut through the shadows. Cedric, empowered by the light, charged straight at Selene, sword raised high. “This ends now!”

But just as he reached her, she unleashed a tempest of dark energy that sent him spiraling across the chamber. Collecting himself, Cedric spotted Lyra in a fierce duel of her own, dancing through the shadows like a spark of fire, each arrow finding its mark as she aimed at Selene.

“Ideals will not save you, Valorian knight!” Selene hissed, conjuring shadows around herself like a vortex.

Lyra met Selene's furious gaze, charging forward with tenacity. “Perhaps not, but the bonds between us are our greatest strength! Together, we can bring your darkness to an end!”

Cedric rallied, feeling the energy pulsing from the Order of the Light. The knights united, their hearts and souls intertwined. Together, they formed a shimmering barrier, pushing back Selene's tempestious magic.

“Your shadows cannot defeat our light!” Cedric declared, and with a unified cry, the knights charged, wielding both blades and magic, raging against the trembling darkness.

The struggle continued in an epic clash, each blow resonating with the values they cherished. Cedric pulled strength from the memories of past victories, the warmth of camaraderie fueling his every movement. The light from the orb flared with incredible intensity, engulfing the chamber in a radiant glow.

“This power belongs to those who embrace love, courage, and hope!” Cedric proclaimed loudly as he launched himself into a final assault against Selene.

With one decisive strike, he plunged his sword toward the heart of the dark spell, shattering the shadows. The cavern filled with brilliance, blinding light erupting from the orb, washing over Selene, and dispelling the darkness that had claimed many souls before her.

For the first time, the shadows retreated, yielding to the pure essence of valiant hearts. Selene screamed, the shadows swirling around her, merging with the light until both erupted in a stunning spectacle.

When the light faded, Cedric and his knights found themselves standing in the cavern, now serene and peaceful. The dark sorceress lay defeated, her power extinguished.

As they emerged from the cavern into the warm sunlight, they knew that the shadows had been vanquished—for now. The bond that had grown between Cedric and his knights solidified with the realization that they had faced their greatest fears together.

Return to Valoria was met with thunderous applause. The people embraced them wholeheartedly, grateful for the sacrifice and courage shown in the face of darkness.

Yet Cedric understood that the fight against despair would never truly end. He pledged to continue building the Order of the Light, instilling hope and resilience in every citizen.

As seasons passed, Valoria became a beacon of strength, an example to other kingdoms. Cedric tirelessly taught and mentored, telling tales of bravery and joint strength against the dark forces that lurked in forgotten corners.

And through it all, Lyra remained at his side, steadfast and fierce, embodying the spirit of the order. Together, they would face any challenge, holding on to the light that bound them, their bond as unyielding as the earth under their feet.

In Cedric's heart bloomed a lasting truth: as long as they stood united as one, darkness could never truly extinguish the light of hope.

Then, one fateful day as Cedric indulged in a leisurely stroll through the castle grounds, he caught sight of a radiant light flickering through the trees. He followed the glow, curiosity guiding him as he entered Eldwood, where hope had flourished.

At the heart of the forest stood Aurelia, the Guardian of the Eldwood, her aura shimmering like the night stars above. Recognizing Cedric’s presence, she smiled, warm and knowing. “You’ve done well, Sir Cedric. Valoria thrives under your care, but remember, the journey for peace requires constant vigilance.”

“What new challenges await us, Aurelia?” Cedric asked, pondering her wisdom.

“A new force stirs beyond the mountains, an ancient prophecy entwined with the energies of darkness and light. Seek it out when the time is ripe, for it may hold the key to lasting balance. Ensure that your courage and the bonds you forge remain steadfast,” Aurelia relayed, her words resonating with power.

Feeling the weight of her message, Cedric nodded solemnly. “Thank you, Aurelia. We will remain vigilant and ready for what lies ahead.”

As the sun dipped below the horizon, casting a warm glow over the kingdom, Cedric felt a renewed sense of purpose twine through him. Together, the Order of the Light would face the mysteries that awaited them beyond the horizon, transforming darkness into bonds of unity and hope.

And so, the legacy of Sir Cedric and Lady Lyra continued to flourish. The heartbeat of Valoria pulsed with promise, echoing tales of courage, love, and the unwavering light that would stand tall against the shadows for all time.

The flame of the Order would live on, unwavering against doubt, courage entwined in the marrow of every knight. They had seen darkness, but together, they had chosen to embrace the light. And in that choice, they left behind a legacy that would inspire both present and future generations to cherish hope, embrace love, and never waver in the face of fear.

With Cedric and Lyra leading the way, the dawn of a new era began, where the heart of Valoria remained unfaltering—a beacon shining brightly against the ever-looming shadows, forever warding off despair.

For every moment lived, they carried the tales of the past, lessons of bravery forever etched in their hearts, preparing to face the rising sun, relentless and glorious, illuminating the path ahead.

As the golden rays of dawn painted the skies over Valoria, a palpable sense of anticipation thrummed in the air. The kingdom had prospered since Cedric and Lyra's victory over Selene, the Shadow Weaver, and the establishment of the Order of the Light. However, Cedric could not shake the feeling that the time for complacency was over; the winds whispered of change, and in the depths of Eldwood, a prophecy was awakening.

The Order had flourished, drawing members from all walks of life eager to protect their homeland. Young recruits trained diligently, honing their skills in swordplay and archery while also learning the values of resilience, friendship, and bravery. Cedric and Lyra often led training sessions, guiding the recruits through exercises that tested both body and spirit.

One crisp morning after a particularly spirited training session with the younger knights, Cedric and Lyra gathered the group to discuss their next steps. “Today, we honor not only our training but also the bonds we forge here,” Cedric began, addressing the eager faces before him. “We will embark on a journey beyond Valoria’s borders to seek knowledge of the ancient prophecy Aurelia mentioned. It is time we learn more about what we may face.”

The faces of the young knights lit up with excitement and determination, the flicker of adventure igniting their spirits. Lyra stepped forward, her energy infectious. “This is an opportunity for growth! We’ll learn together and prepare for any challenges that await us. Who’s ready to stand with us on this quest?”

“I am!” a chorus of voices responded, and the excitement morphed into fervent cheers that resonated through the training grounds.

As they prepared for their journey, gathering supplies, armor, and weapons, Cedric found himself reflecting on the bonds they had cultivated. Each recruit was more than just a soldier; they were family, woven together by shared experiences, victories, and losses. It was this unity that bolstered his resolve to protect Valoria at all costs.

Among the recruits, a young man named Roland stood out. He possessed an infectious spirit and remarkable agility in training but lacked confidence in his abilities. Cedric had noticed this and recognized that he needed to be nurtured. “Roland,” Cedric called to him as they packaged their supplies. “You’ll be joining us on this journey. I’d like you to lead the scouting team. I believe in your potential.”

Roland’s eyes widened in surprise, disbelief taking hold. “Me? But what if I fail? What if I’m not good enough?”

Lyra stepped in, placing a reassuring hand on his shoulder. “You’ve trained with us, you’ve shown tenacity and courage, Roland. Remember, all of us have faced fear, but we move forward together. This is your chance to shine.”

With a hesitant nod, Roland accepted the challenge, determination beginning to flicker in his chest.

As the sun began to set, casting hues of orange and purple across the horizon, the knights gathered around Cedric and Lyra for their final briefing. “We leave at dawn,” Cedric declared. “Stay alert and trust one another. The path ahead will be filled with unknowns, but we will face them together.”

With spirits high and resolve hardened, the Order set forth at first light, leaving the familiarity of Valoria behind them. They rode through valleys adorned with wildflowers and crossed babbling brooks, their laughter echoing against the hills. The camaraderie was palpable, fortifying their determination as they ventured into uncharted territories.

Days passed as they journeyed farther from home. As they traversed forests and crossed wide rivers, the air grew denser with mystery and intangible whispers from the ancient lands. Each evening, they shared tales around the campfire, the bonds between them growing richer with every shared story and laugh.

One crisp evening, as they gathered to rest, Cedric noticed a peculiar tension in the air. The flickering flames cast an uneasy glow, and the shadows danced on the edges of their camp. He looked to Lyra, who seemed to sense it as well.

“Something feels amiss, doesn’t it?” Cedric asked, his voice low, barely above a whisper.

“I feel it too,” she replied, scanning the darkened trees lining their campsite. “We should keep watch tonight, just in case.”

The knights nodded, understanding the importance of vigilance. They set up two rotating watches as the moon climbed high into the sky, shrouding the world beneath a blanket of silver.

The night passed quietly until an eerie hush fell over the land. It was broken suddenly by the loud rustle of leaves. Cedric snapped awake, instincts kickstarting as he surged to his feet, summoning the knights to arms.

“Stay alert! There’s something in the trees!” he commanded, eyes scanning the darkness.

Before they could react, shadowy figures emerged from the forest, ethereal beings with glowing eyes, flickering in and out of sight. Whispers filled the air like a chilling wind, weaving a tapestry of fear.

“They’re back,” Lyra said, her face tense. “Those remnants of Malakar’s darkness!”

Cedric gripped his sword tightly and raised it high. “Knights! Stand together! Our light is brighter than any shadow!”

As the figures advanced, the knights organized themselves, forming a protective circle around their camp. Each knight brandished their weapons, rallying their spirits as they prepared to face the ethereal creatures.

Suddenly, a loud howl pierced the night, and the shadowy figures surged forward, melded with a dark magic that felt cold as ice. The air crackled with tension as the knights clashed with the shadows. Swords met shadow, the enemies coiling around them like a smoke cloud, seeking to extinguish the light.

Lyra fought fiercely, arrows flying with pinpoint accuracy, but the dark beings were relentless, swirling around her and trying to envelop her in despair. “Stay focused!” she shouted, directing her arrows toward the eyes of the nearest shadow.

Cedric charged into the fray, his sword glowing with the power of the orb. “Remember the light within! Focus your energy on the bond we share!” he implored, the force of unity empowering each knight around him.

As they fought, the knights drew from the well of their shared courage, the light emanating from their hearts shining brighter. With each defeat of a shadow, it dissolved in a cloud of shimmering dust, the whispers fading into the ether.

In the chaos of battle, Cedric caught sight of Roland, who, despite being engulfed in shadows, stood his ground. “You can do this, Roland! Remember your training!” Cedric encouraged him.

Gathering his resolve, Roland steadied himself, recalling the skills he had honed with Cedric and the others. With newfound courage, he thrust his sword into the heart of the shadow before him. “For Valoria!” he cried, dispelling the dark haze and breaking free of the grip of despair.

Soon, with their spirits ignited, the knights began to closer in and push back the forces of darkness. They became a united front, each swing of their swords punctuated by shouts of defiance.

The shadows, once powerful, began to retreat, struggling against the light emanating from the knights. With one final charge, Cedric and Lyra led the knights forward, combining their powers. “Together!” they shouted, thrusting their weapons forward in unison.

The blinding light erupted, filling the clearing, shining brighter than they had ever imagined. The shadows screamed as they were consumed by the brilliance, each whisper of despair faltering in the face of their unity.

As the light began to fade, Cedric lowered his sword, breathing heavily as the last wisps of shadow dissipated into nothingness. “We did it,” he gasped, looking at the weary but triumphant faces of his knights.

The battlefield stood silent now, the air clear as night reclaimed its serenity. They had faced the remnants of darkness and emerged victorious.

Roland stepped forward, a newfound strength in his stride. “Thank you for believing in me, Sir Cedric. I didn’t think I had it in me.”

Cedric smiled warmly at him. “You proved that courage is not the absence of fear but the resolve to face it. You were brave tonight, Roland. Together, we learn and grow.”

As dawn crept over the horizon, painting the sky in shades of rose and gold, the Order of the Light celebrated their victory while also reflecting on the dangers that still lurked in the unknown. They knew well that the remnants of Malakar’s dark influence were not entirely vanquished; they still felt the undercurrents of shadow weaving through the realm.

With each step toward their journey’s end, Cedric felt strengthened by the lessons learned and the camaraderie shared with his knights. They were not just fellow warriors but a family bonded through trials and triumphs that promised endurance in the face of future challenges.

As they resumed their travels, the group delved deeper into the heart of the lands, where rumors suggested the ancient prophecy lay waiting for a hero worthy of its secrets. Through valleys and over mountains, each stride brought them closer to uncovering the truth.

Days turned into weeks, and the journey grew ever more arduous. The landscapes shifted from lush forests to barren hills, and remnants of past civilizations caught their attention. Ruins crumbling under the weight of time whispered stories of ancient magic, sparking Cedric's curiosity.

On one such day, as they explored a forgotten temple, they stumbled upon murals adorned with vibrant, intricate designs depicting scenes of light battling darkness. One mural caught Cedric’s eye—a woman standing tall, encircled by the light, quelling monstrous shadows. He recognized her as a figure reminiscent of Aurelia.

“What if this is a depiction of our past battles?” Cedric mused aloud.

“I believe it is,” Lyra agreed, tracing the lines with her fingers. “It speaks to the power of unity between light and dark—how they are bound together, always at odds, yet integral to each other’s existence.”

Suddenly, the ground beneath them quaked, an ancient pulse echoing throughout the halls of the temple. Dust fell like rain from the ceiling, and Cedric steadied himself against a wall. “Everyone, stay alert!” he commanded.

The rumbling subsided, revealing a hidden passageway shimmering with golden light, beckoning them closer as if waiting for them to uncover its secrets. With a mix of trepidation and excitement, they entered the chamber.

At its center lay a magnificent pedestal, and upon it rested an ancient tome. Wisps of energy danced around it, pulsing with an arcane glow that compelled them to step closer. Cedric approached cautiously, feeling the weight of destiny pressing upon him.

“This tome could hold knowledge of the prophecy!” he exclaimed, knowing it might contain vital information for their journey.

As he reached out, Lyra cautioned, “Be careful! Such ancient knowledge may come with a price.”

Ignoring her warning, Cedric lifted the tome, its pages fluttering open as if welcoming him. The chamber filled with light, and images swirled before them—visions of light entangled in a fierce duality with shadow.

In a visual spectacle, Cedric saw the story of Valoria unfold—the rise of light, the birth of darkness, battles fought, creatures unleashed. The images crescendoed to a climax, revealing a prophecy warning of a time when darkness would rise again, and only those who embraced both light and shadow could restore balance.

The final vision centered on Cedric and Lyra, their figures entwined with the energy of the heart of Valoria, emanating warmth and unity, shining brighter than any star.

“That’s us!” Cedric said, filled with awe and a sense of purpose. “We are destined to protect the balance. We can’t allow darkness to claim our realm.”

Suddenly, the tome crackled with energy, and it spoke—its ancient voice echoing through the chamber: “Guardians of light and shadow, your destined path unfolds. Embrace the duality within to restore the fractured realms. The choice lies in understanding—the light thrives through acceptance, and the shadows can guide you.”

As the voice faded, the light dimmed, leaving them standing in solemn silence, digesting the weight of the knowledge imparted.

“We are more than just knights; we have become a part of something greater,” Lyra said, the truth settling in her heart.

With the revelation of the prophecy engraved in their minds, Cedric and his knights prepared to return to Valoria. They had so much to share, lessons learned and knowledge gained, but foremost was the understanding that their journey was not just about fighting against darkness but about embracing all aspects of existence.

Upon their return, the kingdom welcomed them back with open arms. Celebrations erupted in the streets, but Cedric felt the call to gather everyone. Summoning his knights, villagers, and leaders of the Order of the Light, he stood before them, the tome clasped in his hands.

“We have faced challenges and overcome darkness, but we’ve also learned that true strength lies in embracing all aspects of ourselves! The prophecy has revealed that we must unify our purpose,” he declared, gazing into the eager faces before him.

“There is much more to the world than mere light and dark; there is growth, understanding, and wisdom. We will continue to train and understand our duality, and through this, Valoria will emerge stronger than ever!”

The crowd roared with approval, excitement rippling through the air as hope ignited anew.

As the days turned into weeks, Valoria flourished, a blend of light and shadow weaving together to create a tapestry of resilience. Cedric and Lyra led the Order in training, combining lessons of strength and understanding, ensuring that each knight learned to face their fears, channel their powers, and honor the light and shadows inherent in themselves.

Together, they prepared for the unknown, embracing the best and worst of what lay ahead, always united, always shining brightly against the dark. And as they stood on the precipice of what was to come, they faced the horizon with strength and courage—a knight and a guardian, bound by hope and woven with the light of Valoria forever shining in their hearts.

Through all they had encountered, they had forged a legacy that would outlast them, echoing through time as a beacon for future generations, whispering of courage and unity, embracing the fluid dance of light and darkness. It was their journey to protect and nurture, assuring that Valoria thrived amidst the shadows that could never truly extinguish the light, forever forging onward.
""""Here is a longer story to summarize:\n"
    ),

    # 3rd prompt where the LLM's output should contain 1000 tokens
    (
        "Please generate a longer version of the following story in exactly 1000 tokens:\n"
        "In the land of Mystoria, there existed a magical forest known as Everwood. This forest was home to many fantastic creatures, including fairies, talking animals, and even gentle giants. "
        "For centuries, the people of Mystoria lived in harmony with the creatures of Everwood, respecting the balance of nature and the enchantments that kept the forest thriving. "
        "However, as time passed, a dark shadow began to loom over the land. A greedy sorceress named Morgath sought to harness the ancient magic of the forest to gain power and wealth. "
        "She devised a cunning plan to trap the forest's magic and cast a dark curse upon Mystoria. To do this, she enchanted a crystal that would draw magical energy from Everwood. "
        "The creatures of the forest sensed the impending danger and called upon a brave young girl named Lila, who had always felt a deep connection to nature. "
        "Lila, possessing a kind heart and a fierce spirit, agreed to help. She embarked on a journey to gather allies: a wise old owl named Orion, a courageous squirrel named Nibbles, and a gentle giant named Bramble. "
        "Together, they journeyed deep into Everwood, navigating enchanted paths and overcoming obstacles set by Morgath. Each had unique abilities that complemented one another, helping to unveil secrets and unlock the magic of Everwood. "
        "As they neared Morgath's lair, they encountered challenges that tested their unity and courage. Morgath's minions, formed from dark shadows, tried to thwart their progress, but with teamwork and clever strategies, Lila and her friends prevailed. "
        "Finally, they reached the sorceress's lair, where the crystal pulsed with dark energy. A fierce battle ensued as Lila confronted Morgath, who underestimated the strength of friendship and love for nature. "
        "In a climactic moment, Lila used her bond with the forest to channel pure magic, breaking the curse and freeing Everwood from Morgath's grasp. The sorceress was vanquished, and the land rejoiced as nature began to heal. "
        "As a token of gratitude, the creatures of Everwood gifted Lila with a pendant that shimmered with natural magic, ensuring that she would always remain connected to them. "
        "From that day on, Lila became the guardian of Everwood, teaching future generations the importance of protecting nature and living in harmony with its magic."
    ),

    # 4th prompt where the LLM's output should contain approximately 5000 tokens
    (
        "Generate a story that is having 5000 toknes"
    ),

    # 5th prompt for summarization
    (
        "Please summarize the following paragraph:\n"
        "Artificial intelligence has rapidly evolved in recent years, transforming various industries and daily life. From machine learning to natural language processing, AI technologies have allowed for improved efficiency, smarter decision-making, and enhanced customer experiences. Companies are leveraging AI to analyze vast amounts of data and derive insights that were previously unattainable. As AI continues to advance, ethical considerations become increasingly important, prompting discussions around the implications of AI systems in society. It is crucial to balance innovation with responsibility to ensure that the benefits of AI are realized while minimizing potential risks and biases."
    )

]

# Run the evaluation
evaluation_results = evaluate_models_with_prompts(prompts_to_test)

# Print the results
for result in evaluation_results:
    print(f"Prompt: {result['prompt'][:60]}... (Input Characters: {result['input_characters']}, Input Tokens: {result['input_tokens']})")
    print(f"Azure OpenAI - Time: {result['azure_time']:.2f}s, Characters generated: {result['azure_characters']}, Tokens generated: {result['azure_tokens']}")
    print(f"Llama 3 (AWS) - Time: {result['llama_time']:.2f}s, Characters generated: {result['llama_characters']}, Tokens generated: {result['llama_tokens']}")
    print("-" * 50)

Prompt: Summarize the following story - In the heart of the enchante... (Input Characters: 8834, Input Tokens: 2209)
Azure OpenAI - Time: 2.46s, Characters generated: 1206, Tokens generated: 302
Llama 3 (AWS) - Time: 6.32s, Characters generated: 2579, Tokens generated: 645
--------------------------------------------------
Prompt: Summarize the following story - In the faraway kingdom of Va... (Input Characters: 48829, Input Tokens: 12208)
Azure OpenAI - Time: 5.10s, Characters generated: 2049, Tokens generated: 513
Llama 3 (AWS) - Time: 7.68s, Characters generated: 2670, Tokens generated: 668
--------------------------------------------------
Prompt: Please generate a longer version of the following story in e... (Input Characters: 2377, Input Tokens: 595)
Azure OpenAI - Time: 10.42s, Characters generated: 5020, Tokens generated: 1255
Llama 3 (AWS) - Time: 5.63s, Characters generated: 2581, Tokens generated: 646
--------------------------------------------------
Prompt: Generate a sto